In [1]:
import pickle, json, time, datetime, copy, pymongo, re, spacy, scispacy, requests, ast, \
en_ner_bc5cdr_md, en_ner_bionlp13cg_md, en_ner_jnlpba_md, string, os, random
from pymongo import MongoClient
import pandas as pd
from collections import Counter
from tqdm.notebook import tqdm
tqdm.pandas()
from math import ceil
from itertools import combinations, permutations
from spacy.tokenizer import Tokenizer
from multiprocessing import Process, Manager, Pool
from lxml import etree
from nltk.tokenize import sent_tokenize

/root/miniconda3/envs/hald/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
now=datetime.datetime.now()

In [3]:
now_time=now.strftime('%Y_%m_%d')

In [4]:
now_time='2023_09_30'

In [5]:
def cal_time(time):
    time = time.split('/')
    year = int(time[0])
    month = int(time[1]) - 3
    day = int(time[2])
    if month < 1:
        month = 12
        year -= 1
    time = datetime.datetime(year=year, month=month, day=1).strftime('%Y/%m/%d')
    return time

In [6]:
client = MongoClient(username="hald",password="mclab236")

In [7]:
db = client.hald

In [12]:
raw_pick=pickle.load(open('results/step1/raw_pick.pkl','rb'))

In [13]:
raw_pick_pubtator=pickle.load(open('results/step1/raw_pick_pubtator.pkl','rb'))

In [14]:
pubtator_raw=pickle.load(open('results/step1/pubtator_raw.pkl','rb'))

In [15]:
pubtator_raw_dict2AB={}
for i in pubtator_raw:
    pubtator_raw_dict2AB[i.get('id')]=i.get('AB')

# Gene_raw

In [16]:
collections_Gene_raw=db.Gene_raw

In [17]:
rawpickwithAB={}
for i in raw_pick_pubtator:
    rawpickwithAB[i.get('PMID')]=i

In [18]:
pubtator_gene=[]
for i in pubtator_raw:
    if i.get('type')=='Gene':
        pubtator_gene.append(i)

In [19]:
len(pubtator_gene)

36811

In [20]:
Gene2ncbi=pd.read_csv('../ncbi/Gene_ncbi.txt',sep='\t',encoding='utf-8',converters = {'NCBI Gene ID(supplied by NCBI)':str})

In [21]:
ncbi2Gene_dict=dict(zip(Gene2ncbi['NCBI Gene ID(supplied by NCBI)'],Gene2ncbi['Approved symbol'] ))

In [22]:
error=[]
pubtator_genes=[]
for i in pubtator_gene:
    if ncbi2Gene_dict.get(i.get('identifier')):
        i['entity']=ncbi2Gene_dict.get(i.get('identifier'))
        pubtator_genes.append(i)
    else:
        error.append(i)

In [23]:
len(pubtator_genes)

30282

In [24]:
Genelist=pickle.load(open('../dict/Genedict_list.pkl','rb'))

In [25]:
Gene_list={}
for i in Genelist:
    Gene_list[i.lower()]=i.lower()

In [26]:
filtered_pubtator_genes=[]
drop_genes=[]
for i in pubtator_genes:
    if Gene_list.get(i.get('text').lower()):
        filtered_pubtator_genes.append(i)
    else:
        drop_genes.append(i)

In [27]:
len(drop_genes)

7429

In [28]:
len(filtered_pubtator_genes)

22853

In [29]:
filtered_pubtator_genes[10]

{'id': '37031726',
 'AB': "Amyloid precursor protein profusion in Trisomy 21, also called Down Syndrome (DS), is rooted in the genetic determination of Alzheimer's disease (AD). With the recent development in patient care, the life expectancy of DS patients has gradually increased, leading to the high prospect of AD development, consequently leading to the development of plaques of amyloid proteins and neurofibrillary tangles made of tau by the fourth decade of the patient leading to dementia. The altered gene expression resulted in cellular dysfunction due to impairment of autophagy, mitochondrial and lysosomal dysfunction, and copy number variation controlled by the additional genes in Trisomy 21. The cognitive impairment and mechanistic insights underlying DS-AD conditions have been reviewed in this article. Some recent findings regarding biomarkers and therapeutics of DS-AD conditions were highlighted in this review.",
 'type': 'Gene',
 'identifier': '4137',
 'text': 'tau',
 'sente

In [30]:
for i in filtered_pubtator_genes:
    i['ner']='HALD_Gene'
    i['PMID']=i.get('id')
    i['word']=i.get('text')

In [31]:
ner_bc5c = en_ner_bc5cdr_md.load()
ner_bionl = en_ner_bionlp13cg_md.load()
ner_jnlp = en_ner_jnlpba_md.load()

In [32]:
def get_entities(ti, model, keep=[]):
    doc = model(ti)
    if len(keep) > 0:
        return [(x.text.replace('\n', ''), x.label_) for x in doc.ents if str(x.label_) in keep]
    return [(x.text.replace('\n', ''), x.label_) for x in doc.ents ]

In [33]:
raw_pick_pubtator_pmid=[]
for i in raw_pick_pubtator:
    if i.get('AB'):
        raw_pick_pubtator_pmid.append(i.get('PMID'))
    else:
        pass

In [34]:
filtered_pubtator_genes[0]

{'id': '37254731',
 'AB': "Hemophilia, a congenital coagulopathy characterized by a deficiency in coagulation factor VIII (hemophilia A) or factor IX (hemophilia B), results in a tendency of bleeding proportional to the lacking factor. Most bleeds in patients with hemophilia occur in their joints and muscles, and because of these bleeding episodes, patients may end up developing musculoskeletal alterations resulting from hemophilic arthropathy, even receiving hemostatic treatment. The third edition of the World Federation of Hemophilia's Guidelines for the Management of Hemophilia defines 12 principles that encompass all the different types of multidisciplinary care required by people with hemophilia, one of these being clinical and epidemiological research. The expected number of patients with hemophilia across the world has been estimated at 1125 000, most of them undiagnosed, and the total number of patients with severe hemophilia at 418 000 males. Data available from medium and hig

prepare_for_scispacy={}
for i in filtered_pubtator_genes:
    if not filtered_pubtator_genes.get(i.get('PMID')):
        filtered_pubtator_genes[i.get('PMID')]=i

In [35]:
def work(i,finallist_tags,lock):
    finallist_tags.append({'PMID':i.get('PMID'), 'AB':i.get('AB'), 'sentence':i.get('sentence'), 'sentenceIndex': i.get('sentenceIndex'),\
                          'sentenceSize':i.get('sentenceSize'),'word': i.get('word'),'identifier':i.get('identifier'),'type':i.get('type'),\
                           'entity':i.get('entity'),'jnlpba_tags':get_entities(i.get('sentence'), model=ner_jnlp, keep=['DNA','RNA','PROTEIN']),\
                          'bc5cdr_tags':get_entities(i.get('sentence'), model=ner_bc5c),\
                          'bionlp13cg_tags':get_entities(i.get('sentence'), model=ner_bionl, \
                          keep=['GENE_OR_GENE_PRODUCT'])})
if __name__=='__main__':
    start=datetime.datetime.now()
    print('Parent process %s.' % os.getpid())
    p=Pool(60)
    manager=Manager()
    finallist_tags=manager.list()
    lock=manager.Lock()
    for i in filtered_pubtator_genes:
        p.apply_async(work, args=(i,finallist_tags,lock))
    print('Waiting for all subprocesses done...')
    p.close()
    p.join()
    print('All subprocesses done.')
    end=datetime.datetime.now()
    print("The running time is "+str((end-start).seconds)+"s")  

Parent process 40539.
Waiting for all subprocesses done...
All subprocesses done.
The running time is 86s


In [36]:
Gene_tags=list(finallist_tags)

In [37]:
len(Gene_tags)

22853

In [38]:
scispicy_Gene=copy.deepcopy(Gene_tags)

In [39]:
for i in scispicy_Gene:
    i['ScispaCy']=[]
    if i.get('jnlpba_tags'):
        for j in i.get('jnlpba_tags'):
            if j[1]=='DNA' or j[1]=='PROTEIN':
                i['ScispaCy']+=[j[0]]
    if i.get('bionlp13cg_tags'):
        for k in i.get('bionlp13cg_tags'):
            i['ScispaCy']+=[k[0]]
    i['ScispaCy']=list(set(i.get('ScispaCy')))

In [40]:
scicpacy_Genes=[]
not_scicpacy_Genes=[]
for i in scispicy_Gene:
    if i.get('ScispaCy'):
        scicpacy_Genes.append(i)
    else:
        not_scicpacy_Genes.append(i)

In [41]:
len(scicpacy_Genes)

19870

In [42]:
len(not_scicpacy_Genes)

2983

In [43]:
not_scicpacy=[]
for i in not_scicpacy_Genes:
    not_scicpacy.append({'PMID':i.get('PMID'),'word':i.get('word'),'sentence':i.get('sentence')})

In [44]:
not_scicpacy[-10:-1]

[{'PMID': '20956313',
  'word': 'CD8',
  'sentence': 'If infected cell lifespan increased by 15 min because of CD8(+) lymphocyte decay, then there was potential for a thousandfold increase in the number of infected cells.'},
 {'PMID': '23683644',
  'word': 'aid',
  'sentence': 'We aimed to provide global estimates of aid to reproductive health, to assess the allocation of resources across reproductive health activities, and to encourage donor accountability in targeting aid flows to those most in need.'},
 {'PMID': '23683644',
  'word': 'aid',
  'sentence': 'Tracking of donor aid to reproductive health should continue to allow investigation of the allocation of resources across reproductive health activities, and to encourage donor accountability in targeting aid flows to those most in need.'},
 {'PMID': '12065407',
  'word': 'p16',
  'sentence': 'Analyses of the resultant gene products imply that the cells are p16(INK4a) deficient but express physiologically relevant levels of a frame

In [45]:
scicpacy_Genes_final=[]
drop_scispacy=[]
for i in scicpacy_Genes:
    pattern=i.get('word').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                              "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
    for j in i.get('ScispaCy'):
        if i.get('word')==j or re.search(pattern, j):
            scicpacy_Genes_final.append(i)
            break
    else:
        drop_scispacy.append(i)

In [46]:
len(scicpacy_Genes_final)

17863

In [47]:
for i in scicpacy_Genes_final:
    if re.search('-',i.get('word')):
        print(i)
        break

{'PMID': '34480380', 'AB': 'Mucopolysaccharidosis type I (MPS I) is a lysosomal storage disorder characterized by alpha-L-iduronidase deficiency. Patients present with a broad spectrum of disease severity ranging from the most severe phenotype (Hurler) with devastating neurocognitive decline, bone disease and early death to intermediate (Hurler-Scheie) and more attenuated (Scheie) phenotypes, with a normal life expectancy. The most severely affected patients are preferably treated with hematopoietic stem cell transplantation, which halts the neurocognitive decline. Patients with more attenuated phenotypes are treated with enzyme replacement therapy. There are several challenges to be met in the treatment of MPS I patients. First, to optimize outcome, early recognition of the disease and clinical phenotype is needed to guide decisions on therapeutic strategies. Second, there is thus far no effective treatment available for MPS I bone disease. The pathophysiological mechanisms behind bon

In [48]:
scicpacy_Genes_final[0]

{'PMID': '37119829',
 'AB': 'BACKGROUND: Childhood-onset chronic kidney disease is a progressive condition that can have a major effect on life expectancy and quality. We evaluated the usefulness of the kidney tubular cell stress marker urinary Dickkopf-related protein 3 (DKK3) in determining the short-term risk of chronic kidney disease progression in children and identifying those who will benefit from specific nephroprotective interventions. METHODS: In this observational cohort study, we assessed the association between urinary DKK3 and the combined kidney endpoint (ie, the composite of 50% reduction of the estimated glomerular filtration rate [eGFR] or progression to end-stage kidney disease) or the risk of kidney replacement therapy (ie, dialysis or transplantation), and the interaction of the combined kidney endpoint with intensified blood pressure reduction in the randomised controlled ESCAPE trial. Moreover, urinary DKK3 and eGFR were quantified in children aged 3-18 years wit

In [49]:
len(drop_scispacy)

2007

In [50]:
scicpacy_pmids=[]
for i in scicpacy_Genes_final:
    if i.get('PMID') not in scicpacy_pmids:
        scicpacy_pmids.append(i.get('PMID'))

In [51]:
len(scicpacy_pmids)

4454

In [52]:
def query_raw_id(ids, url="https://bern.korea.ac.kr/pubmed/"):
    return requests.get(url + ','.join(ids), verify=False).json()

In [53]:
os.makedirs('../bern/'+now_time)

In [54]:
f=open('../bern/'+now_time+'/download.txt','a')
bern_download_id=1
for j in range(0,len(scicpacy_pmids),5):
    f.write('wget https://bern.korea.ac.kr/pubmed/'+','.join(scicpacy_pmids[j:j+5])+' --no-check-certificate -O '+'../bern/'+now_time+'/bern'+str(bern_download_id)+'.json \n')
    bern_download_id+=1
f.close()

In [55]:
f=open('../bern/'+now_time+'/run.sh','w')
f.write("while read rows; do $rows  ; done < ../bern/"+now_time+'/download.txt')
f.close()

In [56]:
print("nohup bash "+'../bern/'+now_time+'/run.sh > '+'../bern/'+now.strftime('%Y_%m_%d')+'/bern.log &')

nohup bash ../bern/2023_09_30/run.sh > ../bern/2023_10_16/bern.log &


## RUN BERN ON LINUX!!!!!!
### Code is showed above, run it in the current directory.
## Check the file size!!!!!!

all_json=[]
for i in os.listdir('../bern/'+now_time+'/'):
    if i.split('.')[-1] == 'json':
        all_json+=json.load(open('../bern/'+now_time+'/'+i))

allbern=copy.deepcopy(all_json)

len(allbern)

bern_json={}
for i in allbern:
    bern_value=[]
    if i.get('denotations'):
        for j in i.get('denotations'):
            if j.get('obj')=='gene' or j.get('obj')=='protein':
                bern_value.append(i.get('text')[j.get('span').get('begin'):j.get('span').get('end')])
    bern_json[i.get('sourceid')]=bern_value    

len(bern_json)

count=0
filtered_bern_json={}
for i,j in bern_json.items():
    if j:
        count+=1
        filtered_bern_json[i]=j
print(count)

list(filtered_bern_json.items())[0]

In [57]:
SciSpacy_raw=copy.deepcopy(scicpacy_Genes_final)

In [58]:
len(SciSpacy_raw)

17863

In [59]:
SciSpacy_raw_dict={}
for i in SciSpacy_raw:
    SciSpacy_raw_dict[i.get('PMID')]=SciSpacy_raw_dict.get(i.get('PMID'),[])+[i]

In [60]:
len(SciSpacy_raw_dict)

4454

cs=0
intersec={}
dropout=[]
for i,j in filtered_bern_json.items():
    if SciSpacy_raw_dict.get(i):
        midd=[]
        for k in SciSpacy_raw_dict.get(i):
            if k.get('word') in j:
                k['bern']=j
                midd.append(k)
            else:
                for m in j:
                    pattern=k.get('word').translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                      "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
                    if re.search(pattern, m):
                        k['bern']=j
                        midd.append(k)
                        break
                else:
                    dropout.append(k)
        intersec[i]=midd
        

In [61]:
list(SciSpacy_raw_dict.items())[1]

('37254731',
 [{'PMID': '37254731',
   'AB': "Hemophilia, a congenital coagulopathy characterized by a deficiency in coagulation factor VIII (hemophilia A) or factor IX (hemophilia B), results in a tendency of bleeding proportional to the lacking factor. Most bleeds in patients with hemophilia occur in their joints and muscles, and because of these bleeding episodes, patients may end up developing musculoskeletal alterations resulting from hemophilic arthropathy, even receiving hemostatic treatment. The third edition of the World Federation of Hemophilia's Guidelines for the Management of Hemophilia defines 12 principles that encompass all the different types of multidisciplinary care required by people with hemophilia, one of these being clinical and epidemiological research. The expected number of patients with hemophilia across the world has been estimated at 1125 000, most of them undiagnosed, and the total number of patients with severe hemophilia at 418 000 males. Data available 

In [62]:
intersec=copy.deepcopy(SciSpacy_raw_dict)

In [63]:
len(intersec)

4454

In [65]:
final_Gene_raw=[]
for i in intersec.values():
    final_Gene_raw+=i

In [66]:
len(final_Gene_raw)

17863

In [67]:
Genes_inte=copy.deepcopy(final_Gene_raw)

In [68]:
Gene2blank={}
for i in Genes_inte:
    Gene2blank[i.get('entity')]=''

In [69]:
len(Gene2blank)

1996

In [70]:
Gene2ncbi=pd.read_csv('../ncbi/Gene_ncbi.txt',sep='\t',encoding='utf-8',converters = {'NCBI Gene ID(supplied by NCBI)':str})

In [71]:
Gene2ncbi_dict=dict(zip(Gene2ncbi['Approved symbol'], Gene2ncbi['NCBI Gene ID(supplied by NCBI)']))

In [72]:
Gene_converter=pickle.load(open('../ncbi/Gene_converter.pkl','rb'))

In [73]:
len(Gene_converter)

5137

In [74]:
Genes_inte_copy=copy.deepcopy(Genes_inte)

In [75]:
rawpickwithAB={}
for raw in raw_pick_pubtator:
    if raw.get('AB'):
        rawpickwithAB[raw.get('PMID')]=raw

In [76]:
for raw in raw_pick_pubtator:
    if raw.get('PMID')=='37149695':
        print(raw)

In [77]:
raw

{'PMID': '26566813',
 'OWN': 'NLM',
 'STAT': 'MEDLINE',
 'DCOM': '20160407',
 'LR': '20181113',
 'IS': '1471-2253 (Electronic) 1471-2253 (Linking)',
 'VI': '15',
 'DP': '2015 Nov 13',
 'TI': 'Palliative patients under anaesthesiological care: a single-centre retrospective study on incidence, demographics and outcome.',
 'PG': '164',
 'LID': '10.1186/s12871-015-0143-4 [doi] 164',
 'AB': "BACKGROUND: While anesthesiologist's involvement in palliative care has been widely researched, extensive data on palliative patients under anesthesiological care in the operating room is missing. This study was performed to assess the incidence, demographics, and outcome of palliative patients under anesthesiological care. METHODS: We conducted a single-center retrospective chart review of all palliative patients under anesthesiological care at a university hospital in 1 year. Patients were classified as palliative if they fulfilled all predefined criteria (a) incurable, life-threatening disease, (b) p

In [404]:
Gene_deduplicated=set()
Gene_deduplicated_count=0
Gene_list_raw=[]
for i in Genes_inte_copy:
    k=i.get('PMID')
    if rawpickwithAB.get(k):
        i['DP']=rawpickwithAB.get(k).get('DP')
        i['AB']=rawpickwithAB.get(k).get('AB')
        i['JT']=rawpickwithAB.get(k).get('JT')
        i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+k
        i['TI']=rawpickwithAB.get(k).get('TI')
        i['target']=i.get('word')
        i['type']='Gene'
        tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
        if tuples not in Gene_deduplicated:
            Gene_deduplicated.add(tuples)
            Gene_list_raw.append(i)
        else:
            Gene_deduplicated_count+=1
            continue

In [405]:
len(Gene_list_raw)

15820

In [80]:
Gene2blank={}
for i in Gene_list_raw:
    Gene2blank[i.get('entity')]=''

In [81]:
len(Gene2blank)

1996

In [82]:
def get_Gene_info(i,final_Genes_inte):
    ncbiid=Gene2ncbi_dict.get(i)
    ncbiurl='https://www.ncbi.nlm.nih.gov/gene/' + ncbiid
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"
    }
    url = 'https://www.ncbi.nlm.nih.gov/gene/'+ncbiid
    res=requests.get(url=url,headers=headers)
    html = etree.HTML(res.content)
    ncbisummary={
    'Official_Symbol' : html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Official")]/following::dd[1]/text()')[0],
    'Official_Full_Name' : html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Official")]/following::dd[1]/text()')[1],
    'HGNC_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Primary source")]/following::dd[1]/a/text()')),
    'HGNC_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Primary source")]/following::dd[1]/a/@href')),
    'Ensembl_ID' :  ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[1]/text()')),
    'Ensembl_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[1]/@href')),
    'MIM_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[2]/text()')),
    'MIM_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[2]/@href')),
    'AllianceGenome_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[3]/text()')),
    'AllianceGenome_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[3]/@href')),
    'NCBI_ID' : ncbiid ,
    'NCBI_URL' : ncbiurl
    }
    try:
        also=''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Also known as")]/following::dd[1]/text()')).replace('HGNC:HGNC:','')
    except:
        also=''
    finally:
        ncbisummary.update({'Also_known_as' : also})
    try:
        also2=''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Summary")]/following::dd[1]/text()'))
    except:
        also2=''
    finally:
        ncbisummary.update({'Summary' : also2})
    final_Genes_inte[i]=ncbisummary
    time.sleep(0.5)

In [83]:
judge=False
p = Pool(10)
manager=Manager()
final_Genes_inte = manager.dict()
for i in Gene2blank.keys():
    if not Gene_converter.get(i):
        p.apply_async(get_Gene_info, args=(i,final_Genes_inte))
        judge=True
p.close()
p.join()
if judge:
    Gene_converter.update(dict(final_Genes_inte))
    pickle.dump(Gene_converter, open('../ncbi/Gene_converter.pkl','wb'))

In [84]:
judge

True

In [85]:
len(Gene_converter)

5618

In [406]:
for i in range(len(Gene_list_raw)):
    Gene_list_raw[i]=dict(Gene_list_raw[i],**Gene_converter.get(Gene_list_raw[i].get('entity')))

In [407]:
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
for i in Gene_list_raw:
    j=i.get('DP')
    if re.search('[S,s]upplement',j,flags=re.M|re.I):
        j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
    j=re.sub('\s?-\s?','-',j)
    j=re.sub('\.','',j)
    j=re.sub('Aout','Aug',j)
    if len(j.split(' '))==3:
        try:
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        except:
            if j.split()[1] in months:
                if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                    j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                        print('***************************************ERROR**********************************************')
                else:
                    j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
            elif j.split()[1] not in months:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                else:
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==2:
        if re.match(r'Sum(mer)?',j.split()[1]):
            date=int(j[:4]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[1]):
            date=int(j[:4]+'0301')
        elif re.match(r'Win(ter)?',j.split()[1]):
            date=int(j[:4]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
            date=int(j[:4]+'0901')
        elif re.match(r'Sum(mer)?',j.split()[0]):
            date=int(j[-4:]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[0]):
            date=int(j[-4:]+'0301')
        elif re.match(r'Win(ter)?',j.split()[0]):
            date=int(j[-4:]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
            date=int(j[-4:]+'0901')
        else:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
            except:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if len(midd)!=2:
                        midd='0'+midd
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                else:
                    print(j)
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==1:
        j=j[:4]
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    else:
        j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
    i['date']=date
    i['year']=int(str(i.get('date'))[:4])

In [408]:
for i in Gene_list_raw:
    i['addtime']=now_time

In [409]:
Gene_raw=copy.deepcopy(Gene_list_raw)

In [410]:
len(Gene_raw)

15820

In [91]:
if len(Gene_raw)!=0:
    collections_Gene_raw.insert_many(Gene_raw)

# Gene_all

In [423]:
Gene_raw_list=list(collections_Gene_raw.find())

In [424]:
len(Gene_raw_list)

125892

In [425]:
for i in Gene_raw_list:
    i['type']='Gene'
    i['JT']=[i.get('JT')]
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]


In [426]:
Gene_mid={}
for i in Gene_raw_list:
    if not Gene_mid.get(i.get("entity")):
        Gene_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Gene_mid.get(i.get('entity')).get('PMID'):

            ind=Gene_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Gene_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Gene_mid[i.get('entity')]['sentence'][ind] = Gene_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Gene_mid[i.get('entity')]['target'][ind] = Gene_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]
        else:
            Gene_mid[i.get("entity")]["PMID"]=Gene_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Gene_mid[i.get("entity")]["sentence"]=Gene_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Gene_mid[i.get("entity")]["AB"]=Gene_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Gene_mid[i.get("entity")]["DP"]=Gene_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Gene_mid[i.get("entity")]["TI"]=Gene_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Gene_mid[i.get("entity")]["target"]=Gene_mid.get(i.get("entity")).get("target")+i.get("target")
            Gene_mid[i.get("entity")]["date"]=Gene_mid.get(i.get("entity")).get("date")+i.get("date")
            Gene_mid[i.get("entity")]["year"]=Gene_mid.get(i.get("entity")).get("year")+i.get("year")
            Gene_mid[i.get("entity")]["JT"]=Gene_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [427]:
Gene_output_json_final=list(Gene_mid.values())

In [428]:
len(Gene_output_json_final)

5618

In [429]:
for i in Gene_output_json_final:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [430]:
Gene_all=copy.deepcopy(Gene_output_json_final)

# RNA_raw

In [99]:
collections_RNA_raw=db.RNA_raw

In [100]:
names=locals()

In [101]:
raw_pick=pickle.load(open('results/step1/raw_pick.pkl','rb'))

In [102]:
raw_pick_pubtator=pickle.load(open('results/step1/raw_pick_pubtator.pkl','rb'))

In [103]:
pubtator_raw=pickle.load(open('results/step1/pubtator_raw.pkl','rb'))

In [104]:
pubtator_gene=[]
for i in pubtator_raw:
    if i.get('type')=='Gene':
        pubtator_gene.append(i)

In [106]:
os.makedirs('../microRNA_lncRNA_rules/'+now_time)

In [107]:
with open('../microRNA_lncRNA_rules/'+now_time+'/allsentences.txt','a') as f:
    for i in raw_pick_pubtator:
        f.write(i.get('AB')+'\n'+'\n')

In [108]:
with open('../microRNA_lncRNA_rules/'+now_time+'/run.sh','w') as f:
    f.write('''
#!/bin/bash\n
start_time=$(date +%s)
java -cp "../stanza_corenlp/*" -Xmx500g edu.stanford.nlp.pipeline.StanfordCoreNLP -annotators 'tokenize,ssplit,regexner' -file "../microRNA_lncRNA_rules/'''
+now_time+'''/allsentences.txt" -regexner.mapping ../microRNA_lncRNA_rules/jg-regexner.txt -regexner.ignorecase true -tokenize.options "splitHyphenated=false" --outputFormat json --outputDirectory ../microRNA_lncRNA_rules/'''+now_time+'''/  
end_time=$(date +%s)
cost_time=$[ $end_time-$start_time ]
echo "build kernel time is $(($cost_time/60))min $(($cost_time%60))s"
    ''')

In [109]:
os.system("bash ../microRNA_lncRNA_rules/"+now_time+"/run.sh > ../microRNA_lncRNA_rules/"+now_time+"/run.log")

[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator regexner
[main] INFO edu.stanford.nlp.pipeline.TokensRegexNERAnnotator - regexner: Read 21 unique entries out of 22 from ../microRNA_lncRNA_rules/jg-regexner.txt, 0 TokensRegex patterns.

Processing file /root/hald_autoupdate/2023_10_17/../microRNA_lncRNA_rules/2023_09_30/allsentences.txt ... writing to /root/hald_autoupdate/microRNA_lncRNA_rules/2023_09_30/allsentences.txt.json
Annotating file /root/hald_autoupdate/2023_10_17/../microRNA_lncRNA_rules/2023_09_30/allsentences.txt ... done [780.6 sec].

Annotation pipeline timing information:
TokenizerAnnotator: 66.4 sec.
WordsToSentencesAnnotator: 22.4 sec.
TokensRegexNERAnnotator: 691.7 sec.
TOTAL: 780.5 sec. for 7463247 tokens at 9561.5 tokens/sec.
Pipeline setup: 0.5 sec.
Total time for StanfordCoreNLP 

0

In [110]:
jsonfile={}
json_file=json.load(open('../microRNA_lncRNA_rules/'+now_time+'/allsentences.txt.json',"r"))
jsonfile["sentences"]=json_file.get("sentences")

In [111]:
words=[]
entities_dict_list=[]
starttime=datetime.datetime.now()
l=0
HALDnames=[]
start=0
for HALDname in ["LNCRNA","MICRORNA"]:
    HALDnames.append("HALD_"+HALDname)
f=open('../microRNA_lncRNA_rules/'+now_time+'/allsentences.txt')
sentences=f.read()
for i in jsonfile.get("sentences"):
    for HALDname in HALDnames:
        judge=True
        size=len(i.get("tokens"))
        for k,j in enumerate(i.get("tokens")):
            if j.get("ner"):
                if j.get("ner")==HALDname:
                    combination=j.get('word')
                    words.append(combination)
                    entities_dict={}
                    entities_dict["word"]=combination
                    entities_dict["wordBegin"]=j.get("characterOffsetBegin")+start
                    entities_dict["wordEnd"]=j.get("characterOffsetEnd")+start
                    entities_dict["sentenceBegin"]=i.get("tokens")[0].get("characterOffsetBegin")+start
                    entities_dict["sentenceEnd"]=i.get("tokens")[-1].get("characterOffsetEnd")+start
                    entities_dict["sentence"]=sentences[i.get("tokens")[0].get("characterOffsetBegin"):i.get("tokens")[-1].get("characterOffsetEnd")]
                    entities_dict["ner"]=HALDname
                    entities_dict_list.append(entities_dict)
                    if judge:
                        names['l'+HALDname]=k+1
                        judge=False
                    if names['l'+HALDname]<k:
                        names['l'+HALDname]=k+1
                    if k != names['l'+HALDname] -1:
                        continue
                    if names['l'+HALDname]<size:
                        if i.get("tokens")[names['l'+HALDname]].get("ner"):
                            if i.get("tokens")[names['l'+HALDname]].get("ner")==HALDname:
                                while i.get("tokens")[names['l'+HALDname]].get("ner")==HALDname:
                                    combination+=" "+i.get("tokens")[names['l'+HALDname]].get("word")
                                    names['l'+HALDname]+=1
                                    if names['l'+HALDname]>=size:
                                        break
                                names['l'+HALDname]+=1
                                words.append(combination)
                                entities_dict={}
                                entities_dict["word"]=combination
                                entities_dict["wordBegin"]=j.get("characterOffsetBegin")+start
                                entities_dict["wordEnd"]=j.get("characterOffsetEnd")+start
                                entities_dict["sentenceBegin"]=i.get("tokens")[0].get("characterOffsetBegin")+start
                                entities_dict["sentenceEnd"]=i.get("tokens")[-1].get("characterOffsetEnd")+start
                                entities_dict["sentence"]=sentences[i.get("tokens")[0].get("characterOffsetBegin"):i.get("tokens")[-1].get("characterOffsetEnd")]
                                entities_dict["ner"]=HALDname
                                entities_dict_list.append(entities_dict)
f.close()
start+=len(sentences)
endtime=datetime.datetime.now()
print("The running time is "+str((endtime-starttime).seconds)+"s")

The running time is 3s


In [112]:
len(entities_dict_list)

258

In [113]:
entities_dict_list[2]

{'word': 'miR-199a',
 'wordBegin': 1113090,
 'wordEnd': 1113098,
 'sentenceBegin': 1113009,
 'sentenceEnd': 1113162,
 'sentence': 'Aside from that, there was a significant reduction in the expression of miR-100, miR-199a, miR-203, and mir345 in the plasma of OSCC patients (P < 0.05).',
 'ner': 'HALD_MICRORNA'}

In [114]:
length2pmid=[]
start=0
for i in raw_pick_pubtator:
    mid=[]
    mid.append(start)
    mid.append(i.get('PMID'))
    length2pmid.append(mid)
    start+=len(i.get('AB'))+2

In [115]:
len(length2pmid)

27003

In [116]:
length2pmid[10]

[15077, '24882909']

In [117]:
k=0
for i in entities_dict_list:
    for j in range(k,len(length2pmid)):
        if i.get('wordBegin')<length2pmid[j][0]:
            i['PMID']=length2pmid[j-1][1]
            i['AB']=pubtator_raw_dict2AB.get(i.get('PMID'))
            k=j-1
            break
    else:
        i['PMID']=length2pmid[-1][1]
        i['AB']=pubtator_raw_dict2AB.get(i.get('PMID'))

In [118]:
entities_dict_list[10]

{'word': 'miR-485',
 'wordBegin': 1612770,
 'wordEnd': 1612777,
 'sentenceBegin': 1612717,
 'sentenceEnd': 1612910,
 'sentence': 'Moreover, CD44 was predicted as downstream target of miR-485 and the correlation between them was validated by performing dual-luciferase reporter assay and RNA immunoprecipitation (RIP) assay.',
 'ner': 'HALD_MICRORNA',
 'PMID': '30182373',
 'AB': 'Non-small cell lung cancer (NSCLC) remains one of the most aggressive tumors with low life expectancy worldwide. The existence of cancer stem cells (CSCs) contributes to the failure of cancer treatment resulted from drug resistance. Altered microRNA expression has been observed in human tumors due to its role in tumor growth, progression, and metastasis. Hence, the aim of our present study was to investigate the effects of miR-485 on the CSC-like traits in NSCLC A549-cisplatin resistant cells and concentrate on the underlying molecular mechanism. It was found that CSC-like phenotypes were much more enriched in A5

In [119]:
RNA2ncbi=pd.read_csv('../ncbi/RNA_ncbi.txt',sep='\t',encoding='utf-8',converters = {'NCBI Gene ID(supplied by NCBI)':str})

In [120]:
RNA2approved_dict=dict(zip(RNA2ncbi['NCBI Gene ID(supplied by NCBI)'],RNA2ncbi['Approved symbol'] ))

In [121]:
RNA2ncbi_dict=dict(zip(RNA2ncbi['Approved symbol'], RNA2ncbi['NCBI Gene ID(supplied by NCBI)']))

In [122]:
error_RNA=[]
pubtator_RNAs=[]
for i in pubtator_gene:
    if RNA2approved_dict.get(i.get('identifier')):
        i['entity']=RNA2approved_dict.get(i.get('identifier'))
        pubtator_RNAs.append(i)
    else:
        error_RNA.append(i)

In [123]:
len(error_RNA)

36518

In [124]:
len(pubtator_RNAs)

293

In [125]:
names=locals()
HALDnames=[]
for HALDname in ["CLUSTERRNA","LNCRNA","MICRORNA","MISCRNA","RIBOSOMALRNA","SMALL_NUCLEARRNA","SMALL_NUCLEOLARRNA","TRANSFERRNA","VAULTRNA","YRNA"]:
    HALDnames.append("HALD_"+HALDname)
for name in ["clusterRNA","lncRNA","microRNA","miscRNA","ribosomalRNA","small_nuclearRNA","small_nucleolarRNA","transferRNA","vaultRNA","yRNA"]:
    file="../dict/"+name+"/"+name+"dict.json"
    with open(file,"r") as f:
        jsons=json.load(f)
        json_lower={}
        for i,j in jsons.items():
            json_lower[i.lower()]=j
        names[name+"_dict"]=json_lower

In [126]:
pattern=['([Hh][Ss][Aa])?[\-]?[Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa][\-]?\d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'([Hh][Ss][Aa])?[\-]?[Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa] \d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'([Hh][Ss][Aa])?[\-]?[Mm][Ii][Rr][Nn]?[\-]?\d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'([Hh][Ss][Aa])?[\-]?[Mm][Ii][Rr][Nn]? \d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'([Hh][Ss][Aa])?[\-]?([Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa][\-]?\d+)([a-z][/]?)+',
'([Hh][Ss][Aa])?[\-]?[Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa] \d+([a-z][/]?)+',
'([Hh][Ss][Aa])?[\-]?([Mm][Ii][Rr][Nn]?[\-]?\d+)([a-z][/]?)+',
'([Hh][Ss][Aa])?[\-]?[Mm][Ii][Rr][Nn]? \d+([a-z][/]?)+',
'[Hh][Ss][Aa][\-][Mm][Ii][Rr][Nn]?[\-]\d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'([Hh][Ss][Aa][\-][Mm][Ii][Rr][Nn]?[\-]\d+)([a-z][/]?)+',
'[Mm][Ii][Rr][Nn]?[Ll][Ee][Tt]7[A-Ia-i][123]?',
'([Mm][Ii][Rr][Nn]?[Ll][Ee][Tt]7)([A-Ia-i][/]?)+',
'[Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa][\-][Ll][Ee][Tt][\-]\d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'[Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa] [Ll][Ee][Tt] \d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'[Mm][Ii][Rr][Nn]?[\-][Ll][Ee][Tt][\-]\d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'[Mm][Ii][Rr][Nn]? [Ll][Ee][Tt] \d+[a-zA-Z]?[\-]?[12345]?[\-]?[35]?[Pp]?',
'([Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa][\-][Ll][Ee][Tt][\-]\d+)([a-z][/]?)+',
'[Mm][Ii][Cc][Rr][Oo][Rr][Nn][Aa] [Ll][Ee][Tt] \d+([a-z][/]?)+',
'([Mm][Ii][Rr][Nn]?[\-][Ll][Ee][Tt][\-]\d+)([a-z][/]?)+',
'[Mm][Ii][Rr][Nn]? [Ll][Ee][Tt] \d+([a-z][/]?)+']

In [127]:
filtered_pubtator_RNAs=[]
drop_RNAs=[]
midcount1=0
midcount2=0
for i in pubtator_RNAs:
    for name in ["clusterRNA","lncRNA","microRNA","miscRNA","ribosomalRNA","small_nuclearRNA","small_nucleolarRNA","transferRNA","vaultRNA","yRNA"]:
        if names[name+"_dict"].get(i.get('text').lower()):
            i['ner']="HALD_"+name.upper()
            i['type']="RNA"
            i['specific_type']=name.upper()
            filtered_pubtator_RNAs.append(i)
            break
    else:
        for j in pattern:
            if re.search(j,i.get('text').lower()):
                midcount1+=1
                i['ner']="HALD_MICRORNA"
                i['type']="RNA"
                i['specific_type']="MICRORNA"
                filtered_pubtator_RNAs.append(i)
                break
            elif re.search(r'linc\d{5}',i.get('text').lower()):
                midcount2+=1
                i['ner']="HALD_LINCRNA"
                i['type']="RNA"
                i['specific_type']="LINCRNA"
                filtered_pubtator_RNAs.append(i)
                break
        else:
            drop_RNAs.append(i)

In [128]:
print(midcount1)
print(midcount2)

11
0


In [129]:
len(drop_RNAs)

29

In [130]:
filtered_pubtator_genes_pmid2word={}
for i in filtered_pubtator_RNAs:
    filtered_pubtator_genes_pmid2word[i.get('id')]=filtered_pubtator_genes_pmid2word.get(i.get('id'),[])+[i.get('text')]

In [131]:
len(filtered_pubtator_genes_pmid2word)

64

In [132]:
# based on dict. Whether in pubtator
RNA_entities_list=[]
for i in entities_dict_list:
    if filtered_pubtator_genes_pmid2word.get(i.get('PMID')):
        if i.get('word') not in filtered_pubtator_genes_pmid2word.get(i.get('PMID')):
            RNA_entities_list.append(i)
    else:
        RNA_entities_list.append(i)

In [133]:
len(RNA_entities_list)

118

In [134]:
len(entities_dict_list)

258

In [135]:
RNA_entities_list[11]

{'word': 'miR-16',
 'wordBegin': 5430280,
 'wordEnd': 5430286,
 'sentenceBegin': 5430257,
 'sentenceEnd': 5430395,
 'sentence': 'However, expression of miR-16 did not show any statistically significant correlation with age, gender, or expression of apoptotic markers.',
 'ner': 'HALD_MICRORNA',
 'PMID': '31648231',
 'AB': 'Mycosis Fungoides (MF) is the most common type of cutaneous T-cell lymphomas. Early stage patients are treated with topical therapies and have normal life expectancy whereas patients with advanced disease encounter frequent relapses and have a five-year survival rate that does not exceed 15%. The aim of the present study was to characterize the expression of microRNA-16 (miR-16) and microRNA-93 (miR-93) in early and advanced cases of MF in relation to the clinicopathological parameters. Ten skin biopsies of early and advanced MF were investigated for the expression of miR-16 and miR-93 using RT-PCR. Immunohistochemical expression of apoptosis markers (BCL-2 and Surviv

In [136]:
for i in RNA_entities_list:
    i['type']='RNA'
    i['specific_type']=i.get('ner')[5:]

In [137]:
RNAdict=json.load(open('../dict/RNAdict.json','r'))

In [138]:
RNAdict_lower={}
for i,j in RNAdict.items():
    RNAdict_lower[i.lower()]=RNAdict_lower.get(i.lower(),[])+j
    RNAdict_lower[i.lower()]=list(set(RNAdict_lower.get(i.lower())))

In [139]:
cccc=0
final_RNA_entities_list=[]
drop_RNA_entities_list=[]
for i in RNA_entities_list:
    if RNAdict_lower.get(i.get('word').lower()):
        for j in RNAdict_lower.get(i.get('word').lower()):
            i['entity']=j
            final_RNA_entities_list.append(i)
    elif re.search(r'-[3,5]p',i.get('word')):
            if RNAdict_lower.get(re.sub(r'-[3,5]p','',i.get('word'))):
                for j in RNAdict_lower.get(re.sub(r'-[3,5]p','',i.get('word'))):
                    i['entity']=j
                    final_RNA_entities_list.append(i)
                    cccc+=1
    else:
        drop_RNA_entities_list.append(i)

In [140]:
len(final_RNA_entities_list)

14

In [141]:
len(drop_RNA_entities_list)

48

In [142]:
drop_RNA_entities_list[0]

{'word': 'miR-199a',
 'wordBegin': 1113090,
 'wordEnd': 1113098,
 'sentenceBegin': 1113009,
 'sentenceEnd': 1113162,
 'sentence': 'Aside from that, there was a significant reduction in the expression of miR-100, miR-199a, miR-203, and mir345 in the plasma of OSCC patients (P < 0.05).',
 'ner': 'HALD_MICRORNA',
 'PMID': '37114250',
 'AB': "As the most common malignancy, oral squamous cell carcinoma (OSCC) is typically fatal. The survival of patients with oral cancer has not improved, and tumor recurrence remains high. During tumorigenesis, microRNAs (miRNAs) regulate gene expression. Patients' life expectancy can be determined by prognostic survival biomarkers, which can focus therapy on specific targets. This study evaluated five miRNAs associated with OSCC for their prognostic impact. It was determined through microarray analysis and quantitative reverse transcription polymerase chain reaction that there was a significant difference in the expression of miRNAs between OSCC patients an

In [143]:
for i in filtered_pubtator_RNAs:
    i['PMID']=i.get('id')
    i['word']=i.get('text')

In [144]:
final_RNA=filtered_pubtator_RNAs+final_RNA_entities_list

In [145]:
len(final_RNA)

278

In [146]:
RNAs_inte_copy=copy.deepcopy(final_RNA)

In [147]:
deduplicated_RNA=set()
deduplicated_RNA_count=0
RNA_list_raw=[]
for i in RNAs_inte_copy:
    k=i.get('PMID')
    if rawpickwithAB.get(k):
        i['DP']=rawpickwithAB.get(k).get('DP')
        i['AB']=rawpickwithAB.get(k).get('AB')
        i['JT']=rawpickwithAB.get(k).get('JT')
        i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+k
        i['TI']=rawpickwithAB.get(k).get('TI')
        i['target']=i.get('word')
        tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
        if tuples not in deduplicated_RNA:
            deduplicated_RNA.add(tuples)
            RNA_list_raw.append(i)
        else:
            deduplicated_RNA_count+=1
            continue

In [148]:
len(RNA_list_raw)

249

In [149]:
RNA2blank={}
for i in RNA_list_raw:
    RNA2blank[i.get('entity')]=''

In [150]:
len(RNA2blank)

83

In [151]:
RNA_converter=pickle.load(open('../ncbi/RNA_converter.pkl','rb'))

In [152]:
len(RNA_converter)

361

In [153]:
def get_RNA_info(i,final_RNAs_inte):
    ncbiid=RNA2ncbi_dict.get(i)
    ncbiurl='https://www.ncbi.nlm.nih.gov/gene/' + ncbiid
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"
    }
    url = 'https://www.ncbi.nlm.nih.gov/gene/'+ncbiid
    res=requests.get(url=url,headers=headers)
    html = etree.HTML(res.content)
    ncbisummary={
    'Official_Symbol' : html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Official")]/following::dd[1]/text()')[0],
    'Official_Full_Name' : html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Official")]/following::dd[1]/text()')[1],
    'HGNC_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Primary source")]/following::dd[1]/a/text()')),
    'HGNC_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Primary source")]/following::dd[1]/a/@href')),
    'Ensembl_ID' :  ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[1]/text()')),
    'Ensembl_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[1]/@href')),
    'MIM_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[2]/text()')),
    'MIM_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[2]/@href')),
    'AllianceGenome_ID' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[3]/text()')),
    'AllianceGenome_URL' : ''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"See related")]/following::dd[1]/a[3]/@href')),
    'NCBI_ID' : ncbiid ,
    'NCBI_URL' : ncbiurl
    }
    try:
        also=''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Also known as")]/following::dd[1]/text()')).replace('HGNC:HGNC:','')
    except:
        also=''
    finally:
        ncbisummary.update({'Also_known_as' : also})
    try:
        also2=''.join(html.xpath('//*[@id="summaryDl"]/dt[contains(.//text(),"Summary")]/following::dd[1]/text()'))
    except:
        also2=''
    finally:
        ncbisummary.update({'Summary' : also2})
    final_RNAs_inte[i]=ncbisummary
    time.sleep(0.5)

In [154]:
judge=False
p = Pool(10)
manager=Manager()
final_RNAs_inte = manager.dict()
for i in RNA2blank.keys():
    if not RNA_converter.get(i):
        p.apply_async(get_RNA_info, args=(i,final_RNAs_inte))
        judge=True
p.close()
p.join()
if judge:
    RNA_converter.update(dict(final_RNAs_inte))
    pickle.dump(RNA_converter, open('../ncbi/RNA_converter.pkl','wb'))

In [155]:
len(RNA_converter)

385

In [156]:
for i in range(len(RNA_list_raw)):
    RNA_list_raw[i]=dict(RNA_list_raw[i],**RNA_converter.get(RNA_list_raw[i].get('entity')))

In [157]:
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
for i in RNA_list_raw:
    j=i.get('DP')
    if re.search('[S,s]upplement',j,flags=re.M|re.I):
        j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
    j=re.sub('\s?-\s?','-',j)
    j=re.sub('\.','',j)
    j=re.sub('Aout','Aug',j)
    if len(j.split(' '))==3:
        try:
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        except:
            if j.split()[1] in months:
                if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                    j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                        print('***************************************ERROR**********************************************')
                else:
                    j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
            elif j.split()[1] not in months:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                else:
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==2:
        if re.match(r'Sum(mer)?',j.split()[1]):
            date=int(j[:4]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[1]):
            date=int(j[:4]+'0301')
        elif re.match(r'Win(ter)?',j.split()[1]):
            date=int(j[:4]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
            date=int(j[:4]+'0901')
        elif re.match(r'Sum(mer)?',j.split()[0]):
            date=int(j[-4:]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[0]):
            date=int(j[-4:]+'0301')
        elif re.match(r'Win(ter)?',j.split()[0]):
            date=int(j[-4:]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
            date=int(j[-4:]+'0901')
        else:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
            except:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if len(midd)!=2:
                        midd='0'+midd
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                else:
                    print(j)
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==1:
        j=j[:4]
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    else:
        j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
    i['date']=date
    i['year']=int(str(i.get('date'))[:4])

In [158]:
RNA_list_raw[0]

{'id': '37114250',
 'AB': "As the most common malignancy, oral squamous cell carcinoma (OSCC) is typically fatal. The survival of patients with oral cancer has not improved, and tumor recurrence remains high. During tumorigenesis, microRNAs (miRNAs) regulate gene expression. Patients' life expectancy can be determined by prognostic survival biomarkers, which can focus therapy on specific targets. This study evaluated five miRNAs associated with OSCC for their prognostic impact. It was determined through microarray analysis and quantitative reverse transcription polymerase chain reaction that there was a significant difference in the expression of miRNAs between OSCC patients and control patients in plasma. We used the unpaired t-tests and the Mann-Whitney test to conduct the statistical analysis. Based on the study's results, five miRNAs have been found to have significantly different expression levels in the plasma of patients with OSCC; in particular, miR-31 was found to have a signi

In [159]:
for i in RNA_list_raw:
    i['addtime']=now_time

In [160]:
for i,j in RNA_converter.items():
    if j.get('Summary') and j.get('Summary')[0]=='\n':
        print(i)

In [161]:
if len(RNA_list_raw)!=0:
    collections_RNA_raw.insert_many(RNA_list_raw)

# RNA_all

In [162]:
RNA_raw_list=list(collections_RNA_raw.find())

In [163]:
for i in RNA_raw_list:
    i['type']='RNA'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [164]:
RNA_mid={}
for i in RNA_raw_list:
    if not RNA_mid.get(i.get("entity")):
        RNA_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in RNA_mid.get(i.get('entity')).get('PMID'):
            ind=RNA_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in RNA_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                RNA_mid[i.get('entity')]['sentence'][ind] = RNA_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                RNA_mid[i.get('entity')]['target'][ind] = RNA_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]
        else:
            RNA_mid[i.get("entity")]["PMID"]=RNA_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            RNA_mid[i.get("entity")]["sentence"]=RNA_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            RNA_mid[i.get("entity")]["AB"]=RNA_mid.get(i.get("entity")).get("AB")+i.get("AB")
            RNA_mid[i.get("entity")]["DP"]=RNA_mid.get(i.get("entity")).get("DP")+i.get("DP")
            RNA_mid[i.get("entity")]["TI"]=RNA_mid.get(i.get("entity")).get("TI")+i.get("TI")
            RNA_mid[i.get("entity")]["target"]=RNA_mid.get(i.get("entity")).get("target")+i.get("target")
            RNA_mid[i.get("entity")]["date"]=RNA_mid.get(i.get("entity")).get("date")+i.get("date")
            RNA_mid[i.get("entity")]["year"]=RNA_mid.get(i.get("entity")).get("year")+i.get("year")
            RNA_mid[i.get("entity")]["JT"]=RNA_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [165]:
RNA_output_json_final=list(RNA_mid.values())

In [166]:
len(RNA_output_json_final)

385

In [167]:
for i in RNA_output_json_final:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [168]:
RNA_all=copy.deepcopy(RNA_output_json_final)

# chemical_raw

In [169]:
chemical_pubtator_addpmid={}
for i in pubtator_raw:
    if i.get('type')=='Chemical':
        if chemical_pubtator_addpmid.get(i.get('id')):
            chemical_pubtator_addpmid[i.get('id')]=chemical_pubtator_addpmid.get(i.get('id'))+[i]
        else:
            chemical_pubtator_addpmid[i.get('id')]=[i]

In [170]:
print(len(chemical_pubtator_addpmid))

9002


In [171]:
chemical_output_jsons=list(chemical_pubtator_addpmid.values())

In [172]:
chemical_output_json=[]
for i in chemical_output_jsons:
    for j in i:
        chemical_output_json.append(j)

In [173]:
chemical_raw=copy.deepcopy(chemical_output_json)

In [174]:
pairs=['Toxin','Peptide','Lipid','Protein','Carbohydrate','Pharmaceutical Preparations']

In [175]:
names=locals()

In [176]:
alllist=[]
for j in pairs:
    names[j.replace(' ','_').lower()]=pickle.load(open('../chemical/'+j.replace(' ','_')+'/'+j.replace(' ','_')+'.pkl','rb'))
    alllist+=names[j.replace(' ','_').lower()][1:]

In [177]:
alllist[0]

{'text': 'Bacterial Toxins',
 'url': '/mesh/68001427',
 'MeSHUniqueID': 'D001427',
 'type': 'Toxin'}

In [178]:
for i in alllist:
    if not i.get('MeSHUniqueID'):
        i['MeSHUniqueID']='D'+i.get('url')[-6:]

In [179]:
chemicallist=[]
MESHlist=[]
notchemicallist=[]
MESHurl=[]
for i in alllist:
    if i.get('MeSHUniqueID'):
        chemicallist.append({'MESH':'MESH:'+i.get('MeSHUniqueID'),'type':i.get('type'),'url':i.get('url'),'text':i.get('text')})
        MESHlist.append('MESH:'+i.get('MeSHUniqueID'))
        MESHurl.append(i.get('url'))
    else:
        notchemicallist.append(i)

In [180]:
len(notchemicallist)

0

In [181]:
chemical2classdict={}
count=0
for i in chemicallist:
    if not chemical2classdict.get(i.get('MESH')):
        chemical2classdict[i.get('MESH')]=[i.get('type')]
    else:
        if i.get('type') in chemical2classdict.get(i.get('MESH')):
            pass
        else:
            chemical2classdict[i.get('MESH')]=chemical2classdict.get(i.get('MESH'))+[i.get('type')]
            count+=1

In [182]:
chemical2subclassdict={}
for i in chemicallist:
    chemical2subclassdict[i.get('MESH')]=i.get('text')

In [183]:
chemical2urldict={}
for i in chemicallist:
    chemical2urldict[i.get('MESH')]=i.get('url')

In [184]:
for i in chemical_raw:
    i['PMID']=i.get('id')
    i['word']=i.get('text')
    del i['id']
    del i['text']

In [185]:
chemical_final=[]
chemical_mid=[]
chemical_dropout=[]

In [186]:
multi_chemical_type={}
for i in chemical_raw:
    if chemical2classdict.get(i.get('identifier')):
        chemical_final.append(dict(i,**{'class':chemical2classdict.get(i.get('identifier')),'subclass':chemical2subclassdict.get(i.get('identifier')),'url':chemical2urldict.get(i.get('identifier'))}))
        for j in chemical2classdict.get(i.get('identifier')):
            if len(chemical2classdict.get(i.get('identifier')))>1:
                if multi_chemical_type.get(chemical2subclassdict.get(i.get('identifier'))):
                    if j not in multi_chemical_type.get(chemical2subclassdict.get(i.get('identifier'))):
                        multi_chemical_type[chemical2subclassdict.get(i.get('identifier'))]=multi_chemical_type.get(chemical2subclassdict.get(i.get('identifier')))+[j]
                else:
                    multi_chemical_type[chemical2subclassdict.get(i.get('identifier'))]=[j]
    else:
        chemical_dropout.append(i)

In [187]:
len(chemical_final)

5967

In [188]:
multi_chemical_type

{'Sphingomyelins': ['Lipid', 'Carbohydrate'],
 'Exenatide': ['Toxin', 'Peptide'],
 'Micafungin': ['Peptide', 'Lipid'],
 'Glycosphingolipids': ['Lipid', 'Carbohydrate'],
 'Lipopolysaccharides': ['Toxin', 'Lipid', 'Carbohydrate'],
 'Bleomycin': ['Peptide', 'Carbohydrate'],
 'Carnosine': ['Peptide', 'Protein'],
 'Ristocetin': ['Peptide', 'Carbohydrate'],
 'Cilastatin, Imipenem Drug Combination': ['Lipid',
  'Pharmaceutical Preparations'],
 'Vancomycin': ['Peptide', 'Carbohydrate'],
 'Glycolipids': ['Lipid', 'Carbohydrate'],
 'Ceramides': ['Lipid', 'Carbohydrate'],
 'Caspofungin': ['Peptide', 'Lipid'],
 'Myostatin': ['Peptide', 'Protein'],
 'Abciximab': ['Peptide', 'Protein'],
 'Glucosylceramides': ['Lipid', 'Carbohydrate'],
 'Gangliosides': ['Lipid', 'Carbohydrate'],
 'Phalloidine': ['Toxin', 'Peptide'],
 'Lipopeptides': ['Peptide', 'Lipid'],
 'Psychosine': ['Lipid', 'Carbohydrate'],
 'Teicoplanin': ['Peptide', 'Carbohydrate'],
 'G(M2) Ganglioside': ['Lipid', 'Carbohydrate'],
 'Glycopepti

In [189]:
len(chemical_dropout)

35221

In [190]:
for j in pairs:
    names[j.replace(' ','_')+'list']=[]
    for i in chemical_final:
        if j in i.get('class'):
            names[j.replace(' ','_')+'list'].append(i)
    print(len(names[j.replace(' ','_')+'list']))

124
667
2118
628
2719
61


In [191]:
chemical_deduplicated=set()
chemical_deduplicated_count=0
for j in pairs:
    names[j.replace(' ','_')+'list_raw']=[]
    for i in names[j.replace(' ','_')+'list']:
        k=i.get('PMID')
        if rawpickwithAB.get(k):
            i['DP']=rawpickwithAB.get(k).get('DP')
            i['AB']=rawpickwithAB.get(k).get('AB')
            i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+k
            i['TI']=rawpickwithAB.get(k).get('TI')
            i['entity']=i.get('subclass')
            i['target']=i.get('word')
            i['type']=j
            i['JT']=rawpickwithAB.get(k).get('JT')
            if i.get('identifier') and re.match(r'MESH:',i.get('identifier')):
                i['MESH']=i.get('identifier')[5:]
            else:
                i['MESH']=''
            tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"),i.get("type"))
            if tuples not in chemical_deduplicated:
                chemical_deduplicated.add(tuples)
                names[j.replace(' ','_')+'list_raw'].append(i)
            else:
                chemical_deduplicated_count+=1
                continue

In [192]:
chemical2ncbi={}
c=0
for j in pairs:
    for i in names[j.replace(' ','_')+'list_raw']:
        chemical2ncbi[i.get('url')]=''
        c+=1

In [193]:
len(chemical2ncbi)

278

In [194]:
chemical_converter=pickle.load(open('../ncbi/Chemical_converter.pkl','rb'))

In [195]:
len(chemical_converter)

507

In [196]:
def get_chemical_info(i,final_chemicals_inte):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"
    }
    url = 'https://www.ncbi.nlm.nih.gov' + i
    res=requests.get(url=url,headers=headers)
    html = etree.HTML(res.content)
    ncbisummary={
    'Summary' : ''.join(html.xpath('//*[@class="mesh_ds_scope_note"]/text()')),
    'YearIntroduced' : ''.join(html.xpath('//*[@class="mesh_year_introduced"]/text()'))[17:21],
    }
    final_chemicals_inte.update({i:ncbisummary})
    time.sleep(0.5)

In [197]:
judge=False
p = Pool(10)
manager=Manager()
final_chemicals_inte = manager.dict()
for i in chemical2ncbi.keys():
    if not chemical_converter.get(i):
        p.apply_async(get_chemical_info, args=(i,final_chemicals_inte))
        judge=True
p.close()
p.join()
if judge:
    chemical_converter.update(dict(final_chemicals_inte))
    pickle.dump(chemical_converter, open('../ncbi/Chemical_converter.pkl','wb'))

In [198]:
len(chemical_converter)

522

In [199]:
pairs

['Toxin',
 'Peptide',
 'Lipid',
 'Protein',
 'Carbohydrate',
 'Pharmaceutical Preparations']

In [200]:
for j in pairs:
    for i in range(len(names[j.replace(' ','_')+'list_raw'])):
        names[j.replace(' ','_')+'list_raw'][i]=dict(names[j.replace(' ','_')+'list_raw'][i],**chemical_converter.get(names[j.replace(' ','_')+'list_raw'][i].get('url')))

In [201]:
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
for k in pairs:
    for i in names[k.replace(' ','_')+'list_raw']:
        j=i.get('DP')
        if re.search('[S,s]upplement',j,flags=re.M|re.I):
            j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
        j=re.sub('\s?-\s?','-',j)
        j=re.sub('\.','',j)
        j=re.sub('Aout','Aug',j)
        if len(j.split(' '))==3:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
            except:
                if j.split()[1] in months:
                    if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                        j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        try:
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                        except:
                            j=j[:4]
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                            print('***************************************ERROR**********************************************')
                    else:
                        j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                        try:
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                        except:
                            print(j)
                            j=j[:4]
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif j.split()[1] not in months:
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                        if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                            middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                        else:
                            middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                        if middle in months:
                            if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                                j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                            else:
                                j=j.split()[0]+' '+middle+' '+ '1'
                        elif re.match(r'Sum(mer)?',middle):
                            if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                                j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                            else:
                                j=j.split()[0]+' '+'Jun'+' '+ '1'
                        elif re.match(r'Spr(ing)?',middle):
                            if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                                j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                            else:
                                j=j.split()[0]+' '+'Mar'+' '+ '1'
                        elif re.match(r'Win(ter)?',middle):
                            if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                                j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                            else:
                                j=j.split()[0]+' '+'Dec'+' '+ '1'    
                        elif re.match(r'Aut(umn)?',middle) or \
                        re.match(r'Fall',middle):
                            if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                                j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                            else:
                                j=j.split()[0]+' '+'Sep'+' '+ '1'      
                        try:
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                        except:
                            print(j)
                            j=j[:4]
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                    else:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
        elif len(j.split())==2:
            if re.match(r'Sum(mer)?',j.split()[1]):
                date=int(j[:4]+'0601')
            elif re.match(r'Spr(ing)?',j.split()[1]):
                date=int(j[:4]+'0301')
            elif re.match(r'Win(ter)?',j.split()[1]):
                date=int(j[:4]+'1201')
            elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
                date=int(j[:4]+'0901')
            elif re.match(r'Sum(mer)?',j.split()[0]):
                date=int(j[-4:]+'0601')
            elif re.match(r'Spr(ing)?',j.split()[0]):
                date=int(j[-4:]+'0301')
            elif re.match(r'Win(ter)?',j.split()[0]):
                date=int(j[-4:]+'1201')
            elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
                date=int(j[-4:]+'0901')
            else:
                try:
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
                except:
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                        if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                            middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                        else:
                            middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                        if middle in months:
                            j=j.split()[0]+' '+middle+' '+ '1'
                        elif re.match(r'Sum(mer)?',middle):
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                        elif re.match(r'Spr(ing)?',middle):
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                        elif re.match(r'Win(ter)?',middle):
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                        elif re.match(r'Aut(umn)?',middle) or \
                        re.match(r'Fall',middle):
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                        try:
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                        except:
                            j=j[:4]
                            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                    elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                        midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                        if len(midd)!=2:
                            midd='0'+midd
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                    else:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
        elif len(j.split())==1:
            j=j[:4]
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
        else:
            j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        i['date']=date
        i['year']=int(str(i.get('date'))[:4])

In [202]:
for k in pairs:
    names[k.replace(' ','_')+'_raw']=copy.deepcopy(names[k.replace(' ','_')+'list_raw'])
    for i in names[k.replace(' ','_')+'_raw']:
        i['addtime']=now_time

# Carbohydrate_raw

In [203]:
collections_Carbohydrate_raw=db.Carbohydrate_raw

In [204]:
len(Carbohydrate_raw)

2521

In [205]:
Carbohydrate_raw[0]

{'AB': "BACKGROUND: Cancer of ovarian, fallopian tube and peritoneal origin, referred to collectively as ovarian cancer, is the eighth most common cancer in women and is often diagnosed at an advanced stage. Women with relapsed epithelial ovarian cancer (EOC) are less well and have a limited life expectancy, therefore maintaining quality of life with effective symptom control is an important aim of treatment. However, the unwanted effects of chemotherapy agents may be severe, and optimal treatment regimens are unclear. Pegylated liposomal doxorubicin (PLD), which contains a cytotoxic drug called doxorubicin hydrochloride, is one of several treatment modalities that may be considered for treatment of relapsed EOCs. This is an update of the original Cochrane Review which was published in Issue 7, 2013. OBJECTIVES: To evaluate the efficacy and safety of PLD, with or without other anti-cancer drugs, in women with relapsed high grade epithelial ovarian cancer (EOC). SEARCH METHODS: We searc

In [206]:
if len(Carbohydrate_raw)!=0:
    collections_Carbohydrate_raw.insert_many(Carbohydrate_raw)

In [207]:
shenme=list(collections_Carbohydrate_raw.find())

In [208]:
shenme[-1]

{'_id': ObjectId('652d59a2bae5bd33e2c573ce'),
 'AB': 'STUDY OBJECTIVE: To determine whether gallium nitrate therapy is superior to maximally approved doses of calcitonin for acute control of cancer-related hypercalcemia. DESIGN: Randomized, double-blind comparison of active treatments. SETTING: Comprehensive cancer center. PATIENTS: One hundred ninety-eight consecutive hypercalcemic events in 164 patients screened for entry. ELIGIBILITY CRITERIA: hospitalization and intravenous hydration for at least 2 days; persistent elevated serum calcium levels of 2.99 mmol/L or greater (adjusted for serum albumin); serum creatinine levels of 221 mumol/L or less; no cytotoxic chemotherapy, radiation, or mithramycin within the preceding 7 days or during study; no concurrent use of aminoglycoside antibiotics; life expectancy greater than 4 weeks; lymphoma and parathyroid carcinoma excluded. Patients were stratified by histologic type of tumor (epidermoid or nonepidermoid). Fifty patients were randomi

# Carbohydrate_all

In [209]:
Carbohydrate_raw_all=list(collections_Carbohydrate_raw.find())

In [210]:
len(Carbohydrate_raw_all)

20486

In [211]:
for i in Carbohydrate_raw_all:
    i['type']='Carbohydrate'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [212]:
Carbohydrate_mid={}
for i in Carbohydrate_raw_all:
    if not Carbohydrate_mid.get(i.get("entity")):
        Carbohydrate_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Carbohydrate_mid.get(i.get('entity')).get('PMID'):
            ind=Carbohydrate_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Carbohydrate_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Carbohydrate_mid[i.get('entity')]['sentence'][ind] = Carbohydrate_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Carbohydrate_mid[i.get('entity')]['target'][ind] = Carbohydrate_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]
        else:
            Carbohydrate_mid[i.get("entity")]["PMID"]=Carbohydrate_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Carbohydrate_mid[i.get("entity")]["sentence"]=Carbohydrate_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Carbohydrate_mid[i.get("entity")]["AB"]=Carbohydrate_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Carbohydrate_mid[i.get("entity")]["DP"]=Carbohydrate_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Carbohydrate_mid[i.get("entity")]["TI"]=Carbohydrate_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Carbohydrate_mid[i.get("entity")]["target"]=Carbohydrate_mid.get(i.get("entity")).get("target")+i.get("target")
            Carbohydrate_mid[i.get("entity")]["date"]=Carbohydrate_mid.get(i.get("entity")).get("date")+i.get("date")
            Carbohydrate_mid[i.get("entity")]["year"]=Carbohydrate_mid.get(i.get("entity")).get("year")+i.get("year")
            Carbohydrate_mid[i.get("entity")]["JT"]=Carbohydrate_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [213]:
Carbohydrate_all=list(Carbohydrate_mid.values())

In [214]:
len(Carbohydrate_all)

211

In [215]:
for i in Carbohydrate_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [216]:
Carbohydrate_all[10].get('Summary')

'A class of sphingolipids found largely in the brain and other nervous tissue. They contain phosphocholine or phosphoethanolamine as their polar head group so therefore are the only sphingolipids classified as PHOSPHOLIPIDS.'

# Lipid_raw

In [217]:
collections_Lipid_raw=db.Lipid_raw

In [218]:
len(Lipid_raw)

1954

In [219]:
if len(Lipid_raw)!=0:
    collections_Lipid_raw.insert_many(Lipid_raw)

# Lipid_all

In [220]:
Lipid_raw_all=list(collections_Lipid_raw.find())

In [221]:
len(Lipid_raw_all)

21576

In [222]:
for i in Lipid_raw_all:
    if i.get('entity')=='Angiotensins':
        print(i)

In [223]:
for i in Lipid_raw_all:
    i['type']='Lipid'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [224]:
Lipid_mid={}
for i in Lipid_raw_all:
    if not Lipid_mid.get(i.get("entity")):
        Lipid_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Lipid_mid.get(i.get('entity')).get('PMID'):
            ind=Lipid_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Lipid_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Lipid_mid[i.get('entity')]['sentence'][ind] = Lipid_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Lipid_mid[i.get('entity')]['target'][ind] = Lipid_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]
        else:
            Lipid_mid[i.get("entity")]["PMID"]=Lipid_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Lipid_mid[i.get("entity")]["sentence"]=Lipid_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Lipid_mid[i.get("entity")]["AB"]=Lipid_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Lipid_mid[i.get("entity")]["DP"]=Lipid_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Lipid_mid[i.get("entity")]["TI"]=Lipid_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Lipid_mid[i.get("entity")]["target"]=Lipid_mid.get(i.get("entity")).get("target")+i.get("target")
            Lipid_mid[i.get("entity")]["date"]=Lipid_mid.get(i.get("entity")).get("date")+i.get("date")
            Lipid_mid[i.get("entity")]["year"]=Lipid_mid.get(i.get("entity")).get("year")+i.get("year")
            Lipid_mid[i.get("entity")]["JT"]=Lipid_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [225]:
Lipid_all=list(Lipid_mid.values())

In [226]:
len(Lipid_all)

199

In [227]:
for i in Lipid_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

# Peptide_raw

In [228]:
collections_Peptide_raw=db.Peptide_raw

In [229]:
len(Peptide_raw)

643

In [230]:
if len(Peptide_raw)!=0:
    collections_Peptide_raw.insert_many(Peptide_raw)

# Peptide_all

In [231]:
Peptide_raw_all=list(collections_Peptide_raw.find())

In [232]:
len(Peptide_raw_all)

4174

In [233]:
for i in Peptide_raw_all:
    if i.get('entity')=='Angiotensins':
        print(i)
        break

{'_id': ObjectId('64005023648b16758fce3875'), 'AB': 'AIMS: Real-world evidence about heart failure (HF) medical care for elderly patients is limited. The aim of this study was to clarify the age-dependent associations of HF medications at discharge with clinical outcomes in a super-aged society using a real-world big database. METHODS AND RESULTS: Patients hospitalized with a first episode of acute HF in 2013-2019 were identified in a nationwide claims database in Japan and were analyzed based on the HF medications at discharge. Hazard ratios (HRs) and 95 % confidence intervals (95 % CIs) for clinical outcomes were calculated after adjustment. Among 325,468 patients from 4351 hospitals, 130,230 (40.0 %) were aged >= 85 years. All-cause mortality and HF readmission rate within 1 year were 14 % and 23 %, respectively. beta-Blockers and angiotensin-converting-enzyme inhibitors or angiotensin II receptor blockers (ACEI/ARB) uses were associated with lower mortality (HR [95 %CI]: 0.84 [0.83

In [234]:
for i in Peptide_raw_all:
    i['type']='Peptide'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [235]:
Peptide_mid={}
for i in Peptide_raw_all:
    if not Peptide_mid.get(i.get("entity")):
        Peptide_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Peptide_mid.get(i.get('entity')).get('PMID'):

            ind=Peptide_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Peptide_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Peptide_mid[i.get('entity')]['sentence'][ind] = Peptide_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Peptide_mid[i.get('entity')]['target'][ind] = Peptide_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]

        else:
            Peptide_mid[i.get("entity")]["PMID"]=Peptide_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Peptide_mid[i.get("entity")]["sentence"]=Peptide_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Peptide_mid[i.get("entity")]["AB"]=Peptide_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Peptide_mid[i.get("entity")]["DP"]=Peptide_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Peptide_mid[i.get("entity")]["TI"]=Peptide_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Peptide_mid[i.get("entity")]["target"]=Peptide_mid.get(i.get("entity")).get("target")+i.get("target")
            Peptide_mid[i.get("entity")]["date"]=Peptide_mid.get(i.get("entity")).get("date")+i.get("date")
            Peptide_mid[i.get("entity")]["year"]=Peptide_mid.get(i.get("entity")).get("year")+i.get("year")
            Peptide_mid[i.get("entity")]["JT"]=Peptide_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [236]:
Peptide_all=list(Peptide_mid.values())

In [237]:
len(Peptide_all)

89

In [238]:
for i in Peptide_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

# Pharmaceutical_Preparations_raw

In [239]:
collections_Pharmaceutical_Preparations_raw=db.Pharmaceutical_Preparations_raw

In [240]:
len(Pharmaceutical_Preparations_raw)

55

In [241]:
if len(Pharmaceutical_Preparations_raw)!=0:
    collections_Pharmaceutical_Preparations_raw.insert_many(Pharmaceutical_Preparations_raw)

# Pharmaceutical_Preparations_all

In [242]:
Pharmaceutical_Preparations_raw_all=list(collections_Pharmaceutical_Preparations_raw.find())

In [243]:
len(Pharmaceutical_Preparations_raw_all)

327

In [244]:
for i in Pharmaceutical_Preparations_raw_all:
    i['type']='Pharmaceutical Preparations'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [245]:
Pharmaceutical_Preparations_mid={}
for i in Pharmaceutical_Preparations_raw_all:
    if not Pharmaceutical_Preparations_mid.get(i.get("entity")):
        Pharmaceutical_Preparations_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Pharmaceutical_Preparations_mid.get(i.get('entity')).get('PMID'):
            ind=Pharmaceutical_Preparations_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Pharmaceutical_Preparations_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Pharmaceutical_Preparations_mid[i.get('entity')]['sentence'][ind] = Pharmaceutical_Preparations_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Pharmaceutical_Preparations_mid[i.get('entity')]['target'][ind] = Pharmaceutical_Preparations_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]
        else:
            Pharmaceutical_Preparations_mid[i.get("entity")]["PMID"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Pharmaceutical_Preparations_mid[i.get("entity")]["sentence"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Pharmaceutical_Preparations_mid[i.get("entity")]["AB"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Pharmaceutical_Preparations_mid[i.get("entity")]["DP"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Pharmaceutical_Preparations_mid[i.get("entity")]["TI"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Pharmaceutical_Preparations_mid[i.get("entity")]["target"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("target")+i.get("target")
            Pharmaceutical_Preparations_mid[i.get("entity")]["date"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("date")+i.get("date")
            Pharmaceutical_Preparations_mid[i.get("entity")]["year"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("year")+i.get("year")
            Pharmaceutical_Preparations_mid[i.get("entity")]["JT"]=Pharmaceutical_Preparations_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [246]:
Pharmaceutical_Preparations_all=list(Pharmaceutical_Preparations_mid.values())

In [247]:
len(Pharmaceutical_Preparations_all)

15

In [248]:
for i in Pharmaceutical_Preparations_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

# Protein_raw

In [249]:
collections_Protein_raw=db.Protein_raw

In [250]:
len(Protein_raw)

566

In [251]:
if len(Protein_raw)!=0:
    collections_Protein_raw.insert_many(Protein_raw)

# Protein_all

In [252]:
Protein_raw_all=list(collections_Protein_raw.find())

In [253]:
len(Protein_raw_all)

2475

In [254]:
for i in Protein_raw_all:
    if i.get('entity')=='Angiotensins':
        print(i)

{'_id': ObjectId('6400506e648b16758fce4078'), 'AB': 'AIMS: Real-world evidence about heart failure (HF) medical care for elderly patients is limited. The aim of this study was to clarify the age-dependent associations of HF medications at discharge with clinical outcomes in a super-aged society using a real-world big database. METHODS AND RESULTS: Patients hospitalized with a first episode of acute HF in 2013-2019 were identified in a nationwide claims database in Japan and were analyzed based on the HF medications at discharge. Hazard ratios (HRs) and 95 % confidence intervals (95 % CIs) for clinical outcomes were calculated after adjustment. Among 325,468 patients from 4351 hospitals, 130,230 (40.0 %) were aged >= 85 years. All-cause mortality and HF readmission rate within 1 year were 14 % and 23 %, respectively. beta-Blockers and angiotensin-converting-enzyme inhibitors or angiotensin II receptor blockers (ACEI/ARB) uses were associated with lower mortality (HR [95 %CI]: 0.84 [0.83

In [255]:
for i in Protein_raw_all:
    i['type']='Protein'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [256]:
Protein_mid={}
for i in Protein_raw_all:
    if not Protein_mid.get(i.get("entity")):
        Protein_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Protein_mid.get(i.get('entity')).get('PMID'):
            ind=Protein_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Protein_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Protein_mid[i.get('entity')]['sentence'][ind] = Protein_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Protein_mid[i.get('entity')]['target'][ind] = Protein_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]
        else:
            Protein_mid[i.get("entity")]["PMID"]=Protein_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Protein_mid[i.get("entity")]["sentence"]=Protein_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Protein_mid[i.get("entity")]["AB"]=Protein_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Protein_mid[i.get("entity")]["DP"]=Protein_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Protein_mid[i.get("entity")]["TI"]=Protein_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Protein_mid[i.get("entity")]["target"]=Protein_mid.get(i.get("entity")).get("target")+i.get("target")
            Protein_mid[i.get("entity")]["date"]=Protein_mid.get(i.get("entity")).get("date")+i.get("date")
            Protein_mid[i.get("entity")]["year"]=Protein_mid.get(i.get("entity")).get("year")+i.get("year")
            Protein_mid[i.get("entity")]["JT"]=Protein_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [257]:
Protein_all=list(Protein_mid.values())

In [258]:
len(Protein_all)

43

In [259]:
for i in Protein_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

# Toxin_raw

In [260]:
collections_Toxin_raw=db.Toxin_raw

In [261]:
len(Toxin_raw)

123

In [262]:
if len(Toxin_raw)!=0:
    collections_Toxin_raw.insert_many(Toxin_raw)

# Toxin_all

In [263]:
Toxin_raw_all=list(collections_Toxin_raw.find())

In [264]:
len(Toxin_raw_all)

668

In [265]:
for i in Toxin_raw_all:
    i['type']='Toxin'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [266]:
Toxin_mid={}
for i in Toxin_raw_all:
    if not Toxin_mid.get(i.get("entity")):
        Toxin_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Toxin_mid.get(i.get('entity')).get('PMID'):
            ind=Toxin_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Toxin_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Toxin_mid[i.get('entity')]['sentence'][ind] = Toxin_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Toxin_mid[i.get('entity')]['target'][ind] = Toxin_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]
        else:
            Toxin_mid[i.get("entity")]["PMID"]=Toxin_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Toxin_mid[i.get("entity")]["sentence"]=Toxin_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Toxin_mid[i.get("entity")]["AB"]=Toxin_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Toxin_mid[i.get("entity")]["DP"]=Toxin_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Toxin_mid[i.get("entity")]["TI"]=Toxin_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Toxin_mid[i.get("entity")]["target"]=Toxin_mid.get(i.get("entity")).get("target")+i.get("target")
            Toxin_mid[i.get("entity")]["date"]=Toxin_mid.get(i.get("entity")).get("date")+i.get("date")
            Toxin_mid[i.get("entity")]["year"]=Toxin_mid.get(i.get("entity")).get("year")+i.get("year")
            Toxin_mid[i.get("entity")]["JT"]=Toxin_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [267]:
Toxin_all=list(Toxin_mid.values())

In [268]:
len(Toxin_all)

21

In [269]:
for i in Toxin_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

# Mutation_raw

In [270]:
collections_Mutation_raw=db.Mutation_raw

In [271]:
Mutation_pubtator_addpmid={}
for i in pubtator_raw:
    if i.get('type')=='Mutation':
        if Mutation_pubtator_addpmid.get(i.get('id')):
            Mutation_pubtator_addpmid[i.get('id')]=Mutation_pubtator_addpmid.get(i.get('id'))+[i]
        else:
            Mutation_pubtator_addpmid[i.get('id')]=[i]

In [272]:
Mutation_output_json=list(Mutation_pubtator_addpmid.values())

In [273]:
Mutation_raw=sum(Mutation_output_json,[])

In [274]:
len(Mutation_raw)

1511

In [275]:
for i in Mutation_raw:
    i['PMID']=i.get('id')
    i['word']=i.get('text')
    del i['id']
    del i['text']

In [276]:
Mutation_deduplicated=set()
Mutation_deduplicated_count=0
mutation_list_raw=[]
for i in Mutation_raw:
    if i.get('identifier'):
        k=i.get('PMID')
        if rawpickwithAB.get(k):
            i['DP']=rawpickwithAB.get(k).get('DP')
            i['AB']=rawpickwithAB.get(k).get('AB')
            i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+k
            i['TI']=rawpickwithAB.get(k).get('TI')
            i['entity']=i.get('identifier')
            i['target']=i.get('word')
            i['type']='Mutation'
            i['JT']=rawpickwithAB.get(k).get('JT')
            tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
            if tuples not in Mutation_deduplicated:
                Mutation_deduplicated.add(tuples)
                mutation_list_raw.append(i)
            else:
                Mutation_deduplicated_count+=1
                continue

In [277]:
len(mutation_list_raw)

1383

In [278]:
dropout1=[]
dropout2=[]
for i in mutation_list_raw:
    if re.match(r'rs\d+',i.get('identifier')):
        dropout1.append(i)
    else:
        dropout2.append(i)

In [279]:
len(dropout1)

799

In [280]:
len(dropout2)

584

In [283]:
dropout2[0]

{'AB': 'Despite the fact that, in the last years, life expectancy of chronic myeloid leukemia (CML) patients has reached that of the normal population, a significant proportion of CML patients is likely to fail treatment with first- or second-generation tyrosine kinase inhibitors (TKIs). Failure to first-line treatment is commonly due to molecular resistance or unbearable toxicity. New specific compounds are tested in this setting to fulfill this unmet clinical need in CML; of these, asciminib has shown efficacy based on allosteric inhibition which allows to overcome resistance and off-target toxicity. This review aims to cover how asciminib will change the therapeutic scenario of CML, highlighting its mechanism of action, pharmacokinetics, efficacy and toxicity. Asciminib will be a possible option as third-line therapy for patients carrying resistant mutations, such as T315I, and/or not eligible for treatment with other TKIs.',
 'type': 'Mutation',
 'identifier': 'p.T315I',
 'sentence

In [284]:
mutation2ncbi={}
for i in dropout1:
    mutation2ncbi[i.get('entity')]={}

In [285]:
len(mutation2ncbi)

336

In [286]:
mutation_converter=pickle.load(open('../ncbi/Mutation_converter.pkl','rb'))

In [287]:
len(mutation_converter)

1943

In [288]:
def get_mutation_info(i,final_mutations_inte):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"
    }
    url = 'https://www.ncbi.nlm.nih.gov/snp/' + i
    html = etree.HTML(requests.get(url=url,headers=headers).content)
    ncbisummary={
    'Position':''.join(html.xpath('//*[@class="usa-width-one-half"][1]/dd[2]/span[1]/text()'))+''.join(html.xpath('//*[@class="usa-width-one-half"][1]/dd[2]/span[2]/text()')),
    'Alleles':''.join(html.xpath('normalize-space(//*[@class="usa-width-one-half"][1]/dd[3]/text())')),
    'Clinical Significance':'https://www.ncbi.nlm.nih.gov'+''.join(html.xpath('//*[@class="usa-width-one-half"][2]/dd[1]/a/@href')) if html.xpath('//*[@class="usa-width-one-half"][2]/dd[1]/a/@href') else '',
    'Gene : Consequence':[''.join(html.xpath('//*[@class="usa-width-one-half"][2]/dd[2]/div[1]/text()')),''.join(html.xpath('//*[@class="usa-width-one-half"][2]/dd[2]/span[1]/text()'))] if ''.join(html.xpath('//*[@class="usa-width-one-half"][2]/dd[2]/div[1]/text()')) else [''.join(html.xpath('//*[@class="usa-width-one-half"][2]/dd[2]/span[1]/text()'))],
    'Genomic View':url+''.join(html.xpath('//*[@class="usa-width-one-half"][2]/dd[4]/a/@href')),
    }
    final_mutations_inte.update({i:ncbisummary})
    time.sleep(0.5)

In [289]:
judge=False
p = Pool(10)
manager=Manager()
final_mutations_inte = manager.dict()
for i in mutation2ncbi.keys():
    if not mutation_converter.get(i):
        p.apply_async(get_mutation_info, args=(i,final_mutations_inte))
        judge=True
p.close()
p.join()
if judge:
    mutation_converter.update(dict(final_mutations_inte))
    pickle.dump(mutation_converter, open('../ncbi/Mutation_converter.pkl','wb'))

In [290]:
len(mutation_converter)

2204

In [291]:
for i in range(len(mutation_list_raw)):
    if re.match(r'rs\d+',mutation_list_raw[i].get('entity')):
        mutation_list_raw[i]=dict(mutation_list_raw[i],**mutation_converter.get(mutation_list_raw[i].get('entity')))
    else:
        mutation_list_raw[i]=dict(mutation_list_raw[i],**{'Position':'','Alleles':'','Clinical Significance':'','Gene : Consequence':[''],'Genomic View':''})

In [292]:
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
for i in mutation_list_raw:
    j=i.get('DP')
    if re.search('[S,s]upplement',j,flags=re.M|re.I):
        j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
    j=re.sub('\s?-\s?','-',j)
    j=re.sub('\.','',j)
    j=re.sub('Aout','Aug',j)
    if len(j.split(' '))==3:
        try:
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        except:
            if j.split()[1] in months:
                if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                    j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                        print('***************************************ERROR**********************************************')
                else:
                    j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
            elif j.split()[1] not in months:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                else:
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==2:
        if re.match(r'Sum(mer)?',j.split()[1]):
            date=int(j[:4]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[1]):
            date=int(j[:4]+'0301')
        elif re.match(r'Win(ter)?',j.split()[1]):
            date=int(j[:4]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
            date=int(j[:4]+'0901')
        elif re.match(r'Sum(mer)?',j.split()[0]):
            date=int(j[-4:]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[0]):
            date=int(j[-4:]+'0301')
        elif re.match(r'Win(ter)?',j.split()[0]):
            date=int(j[-4:]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
            date=int(j[-4:]+'0901')
        else:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
            except:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if len(midd)!=2:
                        midd='0'+midd
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                else:
                    print(j)
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==1:
        j=j[:4]
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    else:
        j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
    i['date']=date
    i['year']=int(str(i.get('date'))[:4])

In [293]:
len(mutation_list_raw)

1383

In [294]:
mutation_list_raw[0]

{'AB': 'Despite the fact that, in the last years, life expectancy of chronic myeloid leukemia (CML) patients has reached that of the normal population, a significant proportion of CML patients is likely to fail treatment with first- or second-generation tyrosine kinase inhibitors (TKIs). Failure to first-line treatment is commonly due to molecular resistance or unbearable toxicity. New specific compounds are tested in this setting to fulfill this unmet clinical need in CML; of these, asciminib has shown efficacy based on allosteric inhibition which allows to overcome resistance and off-target toxicity. This review aims to cover how asciminib will change the therapeutic scenario of CML, highlighting its mechanism of action, pharmacokinetics, efficacy and toxicity. Asciminib will be a possible option as third-line therapy for patients carrying resistant mutations, such as T315I, and/or not eligible for treatment with other TKIs.',
 'type': 'Mutation',
 'identifier': 'p.T315I',
 'sentence

In [295]:
Mutation_raw=copy.deepcopy(mutation_list_raw)

In [296]:
collections_Mutation_raw.insert_many(Mutation_raw)

# Mutation_all

In [297]:
Mutation_testcopy=list(collections_Mutation_raw.find())

In [298]:
len(Mutation_testcopy)

8470

In [299]:
for i in Mutation_testcopy:
    i['type']='Mutation'
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [300]:
Mutation_mid={}
for i in Mutation_testcopy:
    if not Mutation_mid.get(i.get("entity")):
        Mutation_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Mutation_mid.get(i.get('entity')).get('PMID'):

            ind=Mutation_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Mutation_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Mutation_mid[i.get('entity')]['sentence'][ind] = Mutation_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Mutation_mid[i.get('entity')]['target'][ind] = Mutation_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]
        else:
            Mutation_mid[i.get("entity")]["PMID"]=Mutation_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Mutation_mid[i.get("entity")]["sentence"]=Mutation_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Mutation_mid[i.get("entity")]["AB"]=Mutation_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Mutation_mid[i.get("entity")]["DP"]=Mutation_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Mutation_mid[i.get("entity")]["TI"]=Mutation_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Mutation_mid[i.get("entity")]["target"]=Mutation_mid.get(i.get("entity")).get("target")+i.get("target")
            Mutation_mid[i.get("entity")]["date"]=Mutation_mid.get(i.get("entity")).get("date")+i.get("date")
            Mutation_mid[i.get("entity")]["year"]=Mutation_mid.get(i.get("entity")).get("year")+i.get("year")
            Mutation_mid[i.get("entity")]["JT"]=Mutation_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [301]:
Mutation_all=list(Mutation_mid.values())

In [302]:
for i in Mutation_all:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [303]:
len(Mutation_all)

3224

# Disease_raw

In [304]:
collections_Disease_raw=db.Disease_raw

In [305]:
Disease_pubtator_addpmid={}
for i in pubtator_raw:
    if i.get('type')=='Disease':
        if Disease_pubtator_addpmid.get(i.get('id')):
            Disease_pubtator_addpmid[i.get('id')]=Disease_pubtator_addpmid.get(i.get('id'))+[i]
        else:
            Disease_pubtator_addpmid[i.get('id')]=[i]

In [306]:
Disease_output_json=[]
for i,j in Disease_pubtator_addpmid.items():
    Disease_output_json+=j

In [307]:
len(Disease_output_json)

206881

In [308]:
Disease_mesh=pickle.load(open('../disease/Disease.pkl','rb'))

In [309]:
Disease_mesh=Disease_mesh[1:]

In [310]:
for i in Disease_mesh:
    if i.get('text')=='Uveitis, Suppurative':
        i['MeSHUniqueID']='D015829'

In [311]:
for i in Disease_mesh:
    if i.get('text')=='Hearing Loss':
        print(i)

{'text': 'Hearing Loss', 'url': '/mesh/68034381', 'MeSHUniqueID': 'D034381', 'type': 'Disease'}
{'text': 'Hearing Loss', 'url': '/mesh/68034381', 'MeSHUniqueID': 'D034381', 'type': 'Disease'}
{'text': 'Hearing Loss', 'url': '/mesh/68034381', 'MeSHUniqueID': 'D034381', 'type': 'Disease'}


In [312]:
diseaselist=[]
notdiseaselist=[]
for i in Disease_mesh:
    if i.get('MeSHUniqueID'):
        diseaselist.append({'MESH':'MESH:'+i.get('MeSHUniqueID'),'type':i.get('type'),'url':i.get('url'),'text':i.get('text')})
    else:
        notdiseaselist.append(i)

In [313]:
len(diseaselist)

12791

In [314]:
len(notdiseaselist)

0

In [315]:
disease2classdict={}
count=0
for i in diseaselist:
    if not disease2classdict.get(i.get('MESH')):
        disease2classdict[i.get('MESH')]=[i.get('type')]
    else:
        if i.get('type') in disease2classdict.get(i.get('MESH')):
            pass
        else:
            disease2classdict[i.get('MESH')]=disease2classdict.get(i.get('MESH'))+[i.get('type')]
            count+=1
            if count<=5:
                print(i.get('MESH')+'\t'+str(disease2classdict.get(i.get('MESH'))))

In [316]:
disease2subclassdict={}
count=0
for i in diseaselist:
    disease2subclassdict[i.get('MESH')]=i.get('text')

In [317]:
disease2urldict={}
count=0
for i in diseaselist:
    disease2urldict[i.get('MESH')]=i.get('url')

In [318]:
len(disease2classdict)

4915

In [319]:
for i in Disease_output_json:
    i['PMID']=i.get('id')
    i['word']=i.get('text')
    del i['id']
    del i['text']

In [320]:
Disease_final=[]
Disease_dropout=[]

In [321]:
for i in Disease_output_json:
    if i.get('identifier') in disease2classdict.keys():
        Disease_final.append(dict(i,**{'class':disease2classdict.get(i.get('identifier')),'subclass':disease2subclassdict.get(i.get('identifier')),'url':disease2urldict.get(i.get('identifier'))}))
    else:
        Disease_dropout.append(i)

In [322]:
len(Disease_output_json)

206881

In [323]:
len(Disease_dropout)

21794

In [324]:
Disease_deduplicated=set()
Disease_deduplicated_count=0
disease_list_raw=[]
for i in Disease_final:
    k=i.get('PMID')
    if rawpickwithAB.get(k):
        i['DP']=rawpickwithAB.get(k).get('DP')
        i['AB']=rawpickwithAB.get(k).get('AB')
        i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+k
        i['TI']=rawpickwithAB.get(k).get('TI')
        i['entity']=i.get('subclass')
        i['target']=i.get('word')
        i['type']='Disease'
        i['JT']=rawpickwithAB.get(k).get('JT')
        if i.get('identifier') and re.match(r'MESH:',i.get('identifier')):
            i['MESH']=i.get('identifier')[5:]
        else:
            i['MESH']=''
        # tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
        tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
        if tuples not in Disease_deduplicated:
            Disease_deduplicated.add(tuples)
            disease_list_raw.append(i)
        else:
            Disease_deduplicated_count+=1
            continue

In [325]:
Disease_deduplicated_count

11108

In [326]:
disease2ncbi={}
for i in disease_list_raw:
    disease2ncbi[i.get('url')]=''

In [327]:
len(disease2ncbi)

2464

In [328]:
disease_converter=pickle.load(open('../ncbi/Disease_converter.pkl','rb'))

In [329]:
len(disease_converter)

3359

In [330]:
def get_disease_info(i,final_diseases_inte):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"
    }
    url = 'https://www.ncbi.nlm.nih.gov' + i
    res=requests.get(url=url,headers=headers)
    html = etree.HTML(res.content)
    ncbisummary={
    # 'numbersOfLiterature' : len(i.get('PMID')),
    'Summary' : ''.join(html.xpath('//*[@class="mesh_ds_scope_note"]/text()'))
    }
    final_diseases_inte.update({i:ncbisummary})
    time.sleep(0.5)

In [331]:
judge=False
p = Pool(10)
manager=Manager()
final_diseases_inte = manager.dict()
for i in disease2ncbi.keys():
    if not disease_converter.get(i):
        p.apply_async(get_disease_info, args=(i,final_diseases_inte))
        judge=True
p.close()
p.join()
if judge:
    disease_converter.update(dict(final_diseases_inte))
    pickle.dump(disease_converter, open('../ncbi/Disease_converter.pkl','wb'))

In [332]:
len(disease_converter)

3501

In [333]:
for i in range(len(disease_list_raw)):
    disease_list_raw[i]=dict(disease_list_raw[i],**disease_converter.get(disease_list_raw[i].get('url')))

In [334]:
Disease_deduplicated_dropout=set()
Disease_deduplicated_dropout_count=0
Disease_dropout_raw=[]
for i in Disease_dropout:
    k=i.get('PMID')
    i['DP']=rawpickwithAB.get(k).get('DP')
    i['AB']=rawpickwithAB.get(k).get('AB')
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+k
    i['TI']=rawpickwithAB.get(k).get('TI')
    i['entity']=i.get('word')
    i['target']=i.get('word')
    i['type']='Disease'
    i['JT']=rawpickwithAB.get(k).get('JT')
    if i.get('identifier') and re.match('MESH:',i.get('identifier')):
        i['MESH']=i.get('identifier')[5:]
    else:
        i['MESH']=''
    # tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
    tuples=(i.get("PMID"),i.get("entity"),i.get("word"),i.get("sentence"))
    if tuples not in Disease_deduplicated_dropout:
        Disease_deduplicated_dropout.add(tuples)
        Disease_dropout_raw.append(i)
    else:
        Disease_deduplicated_dropout_count+=1
        continue

In [335]:
for i in Disease_dropout_raw:
    i['Summary']=''

In [345]:
len(Disease_dropout_raw)

20528

In [346]:
Disease_raw=disease_list_raw+Disease_dropout_raw

In [347]:
len(Disease_raw)

194507

In [ ]:
dropout=[]
months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
for i in Disease_raw:
    j=i.get('DP')
    if re.search('[S,s]upplement',j,flags=re.M|re.I):
        j=re.sub('[S,s]upplement.*','',j,flags=re.M|re.I)
    j=re.sub('\s?-\s?','-',j)
    j=re.sub('\.','',j)
    j=re.sub('Aout','Aug',j)
    j=re.sub('[a,A]vril','Apr',j)
    j=re.sub('[f,F]evrier','Feb',j)
    if len(j.split(' '))==3:
        try:
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        except:
            if j.split()[1] in months:
                if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                    j=j.split()[0]+' '+j.split()[1]+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                        print('***************************************ERROR**********************************************')
                else:
                    j=j.split()[0]+' '+j.split()[1]+' '+ '1'
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
            elif j.split()[1] not in months:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+middle+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Jun'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Mar'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Dec'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        if re.search(r'\d+',j.split()[2],flags=re.M|re.I):
                            j=j.split()[0]+' '+'Sep'+' '+ re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
                        else:
                            j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        print(j)
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                else:
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==2:
        if re.match(r'Sum(mer)?',j.split()[1]):
            date=int(j[:4]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[1]):
            date=int(j[:4]+'0301')
        elif re.match(r'Win(ter)?',j.split()[1]):
            date=int(j[:4]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[1]) or re.match(r'Fall',j.split()[1]):
            date=int(j[:4]+'0901')
        elif re.match(r'Sum(mer)?',j.split()[0]):
            date=int(j[-4:]+'0601')
        elif re.match(r'Spr(ing)?',j.split()[0]):
            date=int(j[-4:]+'0301')
        elif re.match(r'Win(ter)?',j.split()[0]):
            date=int(j[-4:]+'1201')
        elif re.match(r'Aut(umn)?',j.split()[0]) or re.match(r'Fall',j.split()[0]):
            date=int(j[-4:]+'0901')
        else:
            try:
                date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b'), '%Y%m'+'01'))
            except:
                if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I) or re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    if re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I):
                        middle=re.search(r'([a-zA-Z]+)[-/]([a-zA-Z]+)',j.split()[1],flags=re.M|re.I).group(1)
                    else:
                        middle=re.search(r'([a-zA-Z]+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if middle in months:
                        j=j.split()[0]+' '+middle+' '+ '1'
                    elif re.match(r'Sum(mer)?',middle):
                        j=j.split()[0]+' '+'Jun'+' '+ '1'
                    elif re.match(r'Spr(ing)?',middle):
                        j=j.split()[0]+' '+'Mar'+' '+ '1'
                    elif re.match(r'Win(ter)?',middle):
                        j=j.split()[0]+' '+'Dec'+' '+ '1'    
                    elif re.match(r'Aut(umn)?',middle) or \
                    re.match(r'Fall',middle):
                        j=j.split()[0]+' '+'Sep'+' '+ '1'      
                    try:
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
                    except:
                        j=j[:4]
                        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
                elif re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I):
                    midd=re.search(r'(\d+)[-/](\d+)',j.split()[1],flags=re.M|re.I).group(1)
                    if len(midd)!=2:
                        midd='0'+midd
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+midd+'01'))
                else:
                    print(j)
                    j=j[:4]
                    date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    elif len(j.split())==1:
        j=j[:4]
        date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y'), '%Y'+'0101'))
    else:
        try:
            notuse=re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
            j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
        except:
            date=int(datetime.datetime.strftime(datetime.datetime.strptime(j[:4],'%Y'), '%Y'+'0101'))
        # j=j.split()[0]+' '+j.split()[1]+' '+re.search(r'\d+',j.split()[2],flags=re.M|re.I).group()
        # date=int(datetime.datetime.strftime(datetime.datetime.strptime(j,'%Y %b %d'), '%Y%m%d'))
    i['date']=date
    i['year']=int(str(i.get('date'))[:4])

In [344]:
j

'2022 Special Issue on Puerto Rico'

In [ ]:
collections_Disease_raw.insert_many(Disease_raw)

# Disease_all

In [ ]:
Disease_raw_list=list(collections_Disease_raw.find())

In [ ]:
for i in Disease_raw_list:
    if i.get('word').upper()=="ALZHEI-MER'S DISEASE":
        i['entity']='Alzheimer Disease'

In [ ]:
for i in Disease_raw_list:
    i["PMID"]=[i.get("PMID")]
    i["sentence"]=[[i.get("sentence")]]
    pmid=i.get('PMID')[0]
    i['DP']=[i.get("DP")]
    i['AB']=[i.get("AB")]
    i['URL']='https://pubmed.ncbi.nlm.nih.gov/'+pmid
    i['TI']=[i.get("TI")]
    i['target']=[[i.get('word')]]
    i["date"]=[i.get("date")]
    i["year"]=[i.get("year")]
    i['JT']=[i.get('JT')]

In [ ]:
i

In [ ]:
Disease_mid={}
for i in Disease_raw_list:
    if not Disease_mid.get(i.get("entity")):
        Disease_mid[i.get("entity")]=i
    else:
        if i.get('PMID')[0] in Disease_mid.get(i.get('entity')).get('PMID'):
            ind=Disease_mid.get(i.get('entity')).get('PMID').index(i.get('PMID')[0])
            if i.get("sentence")[0][0] in Disease_mid.get(i.get('entity')).get('sentence')[ind]:
                continue
            else:
                Disease_mid[i.get('entity')]['sentence'][ind] = Disease_mid.get(i.get('entity')).get('sentence')[ind] + i.get("sentence")[0]
                Disease_mid[i.get('entity')]['target'][ind] = Disease_mid.get(i.get('entity')).get('target')[ind] + i.get("target")[0]
        else:
            Disease_mid[i.get("entity")]["PMID"]=Disease_mid.get(i.get("entity")).get("PMID")+i.get("PMID")
            Disease_mid[i.get("entity")]["sentence"]=Disease_mid.get(i.get("entity")).get("sentence")+i.get("sentence")
            Disease_mid[i.get("entity")]["AB"]=Disease_mid.get(i.get("entity")).get("AB")+i.get("AB")
            Disease_mid[i.get("entity")]["DP"]=Disease_mid.get(i.get("entity")).get("DP")+i.get("DP")
            Disease_mid[i.get("entity")]["TI"]=Disease_mid.get(i.get("entity")).get("TI")+i.get("TI")
            Disease_mid[i.get("entity")]["target"]=Disease_mid.get(i.get("entity")).get("target")+i.get("target")
            Disease_mid[i.get("entity")]["date"]=Disease_mid.get(i.get("entity")).get("date")+i.get("date")
            Disease_mid[i.get("entity")]["year"]=Disease_mid.get(i.get("entity")).get("year")+i.get("year")
            Disease_mid[i.get("entity")]["JT"]=Disease_mid.get(i.get("entity")).get("JT")+i.get("JT")

In [ ]:
Disease_output_json_final=list(Disease_mid.values())

In [ ]:
len(Disease_output_json_final)

In [ ]:
for i in Disease_output_json_final:
    i['allyears']=[]
    i['datestring']=[]
    for j in i.get('date'):
        if int(str(j)[:4]) not in i.get('allyears'):
            i['allyears']= i.get('allyears')+[int(str(j)[:4])]
    for j in i.get('date'):
        if str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:] not in i.get('datestring'):
            i['datestring']=i.get('datestring')+[str(j)[:4]+"-"+str(j)[4:6]+"-"+str(j)[6:]]
    i['allyears']=sorted(i.get('allyears'), reverse=True)
    i['datestring']=sorted(i.get('datestring'), reverse=True)
    upperlist=set()
    for j in sum(i.get('target'),[]):
        upperlist.add(j.upper())
    i['targetshapes']=list(upperlist)
    i['addtime']=now_time

In [ ]:
Disease_all=copy.deepcopy(Disease_output_json_final)

In [ ]:
len(Disease_all)

In [ ]:
names=locals()

In [ ]:
entity_pairs=['Gene','RNA','Carbohydrate','Lipid','Peptide', 'Pharmaceutical_Preparations', 'Protein', 'Toxin','Mutation', 'Disease']

In [ ]:
for i in entity_pairs:
    pickle.dump(names[i+'_all'],open('results/step2/'+i+'_all.pkl','wb'))

In [ ]:
for i in entity_pairs:
    names[i+'_all']=pickle.load(open('results/step2/'+i+'_all.pkl','rb'))

In [ ]:
Gene_all[10].keys()

In [ ]:
RNA_all[10].keys()

## GenAge

In [623]:
Genedict=json.load(open('../dict/Genedict.json'))

In [624]:
Genedict_upper={}
for i,j in Genedict.items():
    temp_list=[]
    for k in j:
        if k not in temp_list:
            temp_list.append(k)
    Genedict_upper[i.upper()]=temp_list

In [625]:
import os
import pandas as pd

import csv
import pickle
import json

In [626]:
genage=pd.read_csv('../other_databases/genage_human.csv')

In [627]:
genage[:2]

,GenAge ID,symbol,name,entrez gene id,uniprot,why
0,1,GHR,growth hormone receptor,2690,GHR_HUMAN,mammal
1,2,GHRH,growth hormone releasing hormone,2691,SLIB_HUMAN,mammal


In [628]:
len(genage)

307

In [630]:
for i,j in genage_symbol_transfer.items():
    if i!=j[0]:
        print(i)
        print(j)
        break

ATP5PO
['ATP5O']


In [631]:
len(genage_symbol)

306

In [632]:
# already check that one to one
genage_symbol=set()
genage_symbol_transfer={}
for i in genage['symbol'].tolist():
    if Genedict_upper.get(i.upper()):
        genage_symbol_transfer[Genedict_upper.get(i.upper())[0]]=i
    else:
        # genage_symbol_transfer[i]=i
        print(i)

TERC


In [633]:
for i in Gene_all:
    if genage_symbol_transfer.get(i.get('entity')):
        i['GenAge_url']='https://genomics.senescence.info/genes/entry.php?hgnc='+genage_symbol_transfer.get(i.get('entity'))
        i['GenAge_symbol']=genage_symbol_transfer.get(i.get('entity'))
    else:
        i['GenAge_url']=''
        i['GenAge_symbol']=''

In [634]:
for i in RNA_all:
    if i.get('entity')=='TERC':
        i['GenAge_url']='https://genomics.senescence.info/genes/entry.php?hgnc='+'TERC'
        i['GenAge_symbol']='TERC'
    else:
        i['GenAge_url']=''
        i['GenAge_symbol']=''

## LongevityMap

In [718]:
RNAdict=json.load(open('../dict/RNAdict.json'))

In [719]:
RNAdict_upper={}
for i,j in RNAdict.items():
    temp_list=[]
    for k in j:
        if k not in temp_list:
            temp_list.append(k)
    RNAdict_upper[i.upper()]=temp_list

In [720]:
longevity_human=pd.read_csv('../other_databases/longevity_human.csv')

In [721]:
len(longevity_human)

550

In [722]:
longevity_human_genes_symbol=set()
for j in longevity_human['Gene(s)'].tolist():
    if type(j)==str:
        for i in j.split(','):
            longevity_human_genes_symbol.add(i)
print(len(longevity_human_genes_symbol))

884


In [723]:
longevity_human_symbol=set()
longevity_human_symbol_transfer={}
for j in longevity_human['Gene(s)'].tolist():
    if type(j)==str:
        for i in j.split(','):
            if Genedict_upper.get(i.upper()) and len(Genedict_upper.get(i.upper()))>1 and i!=Genedict_upper.get(i.upper())[0]:
                print(i)
                print(Genedict_upper.get(i.upper()))
                break

In [724]:
# already check that one to one
longevity_human_symbol=set()
longevity_human_symbol_transfer={}
for j in longevity_human['Gene(s)'].tolist():
    if type(j)==str:
        for i in j.split(','):
            if Genedict_upper.get(i.upper()):
                longevity_human_symbol_transfer[Genedict_upper.get(i.upper())[0]]=i
            elif RNAdict_upper.get(i.upper()):
                longevity_human_symbol_transfer[RNAdict_upper.get(i.upper())[0]]=i
            else:
                print(i)
                break

GSTT1
HLA-DRB3
GSTT1
GSTT1
LOC105375494
LOC107987209
LOC100631378
LOC105378102
GSTT1
LOC107984390
LOC107985064
LOC105378963
LOC101928226
TTC41P
LOC107987043
LOC105378178
LOC105377329
LOC105374956
LOC105370194
CYP2C58P
LOC101928977
LOC105378093
LOC101059906
OR5F2P
LOC400627
CIDECP
LOC105378228


In [729]:
for i in Gene_all:
    if longevity_human_symbol_transfer.get(i.get('entity')):
        i['LongevityMap_url']='https://genomics.senescence.info/longevity/search.php?search='+longevity_human_symbol_transfer.get(i.get('entity'))
        i['LongevityMap_symbol']=longevity_human_symbol_transfer.get(i.get('entity'))
    else:
        i['LongevityMap_url']=''
        i['LongevityMap_symbol']=''

In [730]:
for i in RNA_all:
    if longevity_human_symbol_transfer.get(i.get('entity')):
        i['LongevityMap_url']='https://genomics.senescence.info/longevity/search.php?search='+longevity_human_symbol_transfer.get(i.get('entity'))
        i['LongevityMap_symbol']=longevity_human_symbol_transfer.get(i.get('entity'))
    else:
        i['LongevityMap_url']=''
        i['LongevityMap_symbol']=''

longevity_human_genes_symbol2id={}
for i in range(len(longevity_human)):
    if type(longevity_human.iloc[i,4])==str:
        for j in longevity_human.iloc[i,4].split(','):
            longevity_human_genes_symbol2id[j]=longevity_human_genes_symbol2id.get(j,[])+[longevity_human.iloc[i,0]]

list(longevity_human_genes_symbol2id.items())[3]

for i in Gene_all:
    temp_symbol=[]
    temp_url=[]
    if longevity_human_symbol_transfer.get(i.get('entity')):
        if longevity_human_genes_symbol2id.get(longevity_human_symbol_transfer.get(i.get('entity'))):
            for j in longevity_human_genes_symbol2id.get(longevity_human_symbol_transfer.get(i.get('entity'))):
                temp_symbol.append(j)
                temp_url.append('https://genomics.senescence.info/longevity/entry.php?id='+j)
            i['LongevityMap_url']=temp_url
            i['LongevityMap_symbol']=temp_symbol
    else:
        i['LongevityMap_url']=''
        i['LongevityMap_symbol']=''

for i in RNA_all:
    temp_symbol=[]
    temp_url=[]
    if longevity_human_symbol_transfer.get(i.get('entity')):
        if longevity_human_genes_symbol2id.get(longevity_human_symbol_transfer.get(i.get('entity'))):
            for j in longevity_human_genes_symbol2id.get(longevity_human_symbol_transfer.get(i.get('entity'))):
                temp_symbol.append(j)
                temp_url.append('https://genomics.senescence.info/longevity/entry.php?id='+j)
            i['LongevityMap_url']=temp_url
            i['LongevityMap_symbol']=temp_symbol
    else:
        i['LongevityMap_url']=''
        i['LongevityMap_symbol']=''

## AgingBank

In [648]:
agingbank_genes_pro=pd.read_csv('../other_databases/agingbank_genes_pro.csv')

In [649]:
agingbank_genes_pro[:3]

,Species,Type,name,sample,Tissue,Up/Down,Pro/Anti,Pubmed ID,Details
0,Homo sapiens,gene,ABI3,human,"7 normal thyroid tissues, 21 follicular thyroi...",Down,Pro,21223585,details
1,Homo sapiens,gene,ABI3BP,human,"7 normal thyroid tissues, 21 follicular thyroi...",Down,Pro,21223585,details
2,Homo sapiens,gene,ACE,human,NaN,NaN,Pro,16971231,details


In [650]:
agingbank_genes_pro_list=agingbank_genes_pro['name'].tolist()

In [651]:
len(agingbank_genes_pro_list)

97

# already check that one to one except p21 possibly for ['CDKN1A', 'TCEAL1']
# already check that no RNA
agingbank_genes_pro_symbol=set()
agingbank_genes_pro_symbol_transfer={}
for i in agingbank_genes_pro_list:
    if re.search(r'(.*)(\(.*\))',i):
        i=re.search(r'(.*)(\(.*\))',i).groups()[0]
    if re.search(r'(.*)(\s.*)',i):
        i=re.search(r'(.*)(\s.*)',i).groups()[0]
    if Genedict_upper.get(i.upper()):
        if len(Genedict_upper.get(i.upper()))>1 and i!=Genedict_upper.get(i.upper())[0]:
            print(i)
            print(Genedict_upper.get(i.upper()))
    elif RNAdict_upper.get(i.upper()):
        print(i)
        # if len(RNAdict_upper.get(i.upper()))>1 and i!=RNAdict_upper.get(i.upper())[0]:
        #     print(i)
        #     print(RNAdict_upper.get(i.upper()))

In [652]:
# already check that one to one except p21 possibly for ['CDKN1A', 'TCEAL1']
# already check that no RNA
agingbank_genes_pro_symbol=set()
agingbank_genes_pro_symbol_transfer={}
for i in agingbank_genes_pro_list:
    if re.search(r'(.*)(\(.*\))',i):
        i=re.search(r'(.*)(\(.*\))',i).groups()[0]
    if re.search(r'(.*)(\s.*)',i):
        i=re.search(r'(.*)(\s.*)',i).groups()[0]
    if Genedict_upper.get(i.upper()):
        if len(Genedict_upper.get(i.upper()))>1 and i!=Genedict_upper.get(i.upper())[0]:
            agingbank_genes_pro_symbol_transfer[Genedict_upper.get(i.upper())[0]]=i
            agingbank_genes_pro_symbol_transfer[Genedict_upper.get(i.upper())[1]]=i
        else:
            agingbank_genes_pro_symbol_transfer[Genedict_upper.get(i.upper())[0]]=i
        # agingbank_genes_pro_symbol.add(Genedict_upper.get(i.upper())[0])
#         if len(Genedict_upper.get(i.upper()))>1 and i!=Genedict_upper.get(i.upper())[0]:
            
#         else:
#             agingbank_genes_pro_symbol_transfer[Genedict_upper.get(i.upper())[0]]=[i]

In [653]:
len(agingbank_genes_pro_symbol_transfer)

55

In [654]:
for i in Gene_all:
    if agingbank_genes_pro_symbol_transfer.get(i.get('entity')):
        i['AgingBank_url']='http://bio-bigdata.hrbmu.edu.cn/AgingBank/searchAdvRes.jsp?species=Homo%20sapien'
        i['AgingBank_symbol']=agingbank_genes_pro_symbol_transfer.get(i.get('entity'))
    else:
        i['AgingBank_url']=''
        i['AgingBank_symbol']=''

In [686]:
agingbank_genes_anti=pd.read_csv('../other_databases/agingbank_genes_anti.csv')

In [687]:
agingbank_genes_anti[:3]

,Species,Type,name,sample,Tissue,Up/Down,Pro/Anti,Pubmed ID,Details
0,Homo sapiens,gene,AEG-1,cell line,LNCaP;C42B,Down,Anti,28923415,details
1,Homo sapiens,gene,AKT1,human,Brain,NaN,Anti,16914869,details
2,Homo sapiens,gene,APOE,human,NaN,NaN,Anti,17234815,details


In [690]:
agingbank_genes_anti_list=agingbank_genes_anti['name'].tolist()

In [691]:
len(agingbank_genes_anti)

154

In [692]:
# already check that one to one except p21 possibly for ['CDKN1A', 'TCEAL1']
# already check that no RNA
agingbank_genes_anti_symbol=set()
agingbank_genes_anti_symbol_transfer={}
for i in agingbank_genes_anti_list:
    if re.search(r'(.*)(\(.*\))',i):
        i=re.search(r'(.*)(\(.*\))',i).groups()[0]
    if re.search(r'(.*)(\s.*)',i):
        i=re.search(r'(.*)(\s.*)',i).groups()[0]
    if Genedict_upper.get(i.upper()):
        if len(Genedict_upper.get(i.upper()))>1 and i!=Genedict_upper.get(i.upper())[0]:
            agingbank_genes_anti_symbol_transfer[Genedict_upper.get(i.upper())[0]]=i
            agingbank_genes_anti_symbol_transfer[Genedict_upper.get(i.upper())[1]]=i
        else:
            agingbank_genes_anti_symbol_transfer[Genedict_upper.get(i.upper())[0]]=i
        # agingbank_genes_anti_symbol.add(Genedict_upper.get(i.upper())[0])
#         if len(Genedict_upper.get(i.upper()))>1 and i!=Genedict_upper.get(i.upper())[0]:
            
#         else:
#             agingbank_genes_anti_symbol_transfer[Genedict_upper.get(i.upper())[0]]=[i]

In [693]:
len(agingbank_genes_anti_symbol_transfer)

99

In [779]:
for i in Gene_all:
    if agingbank_genes_anti_symbol_transfer.get(i.get('entity')):
        i['AgingBank_url']='http://bio-bigdata.hrbmu.edu.cn/AgingBank/searchAdvRes.jsp?species=Homo%20sapiens'
        i['AgingBank_symbol']=agingbank_genes_anti_symbol_transfer.get(i.get('entity'))
    else:
        i['AgingBank_url']=''
        i['AgingBank_symbol']=''

## AgingAtlas

In [655]:
agingatlas_genes=pd.read_csv('../other_databases/agingatlas_genes.csv')

In [656]:
agingatlas_genes[:3]

,Symbol
0,IL2RB
1,PSAT1
2,A2M


In [657]:
agingatlas_genes_list=agingatlas_genes['Symbol'].tolist()

In [658]:
len(agingatlas_genes_list)

502

# already check that one to one
# already check that RNA only TERC
agingatlas_genes_symbol=set()
agingatlas_genes_symbol_transfer={}
for i in agingatlas_genes_list:
    if Genedict_upper.get(i.upper()):
        if len(Genedict_upper.get(i.upper()))>1 and i!=Genedict_upper.get(i.upper())[0]:
            print(i)
            print(Genedict_upper.get(i.upper()))
    elif RNAdict_upper.get(i.upper()):
        print(i)
        # if len(Genedict_upper.get(i.upper()))>1 and i!=Genedict_upper.get(i.upper())[0]:
        #     print(i)
        #     print(Genedict_upper.get(i.upper()))
    #     agingatlas_genes_symbol_transfer[Genedict_upper.get(i.upper())[0]]='https://ngdc.cncb.ac.cn/aging/age_related_genes'
    # elif RNAdict_upper.get(i.upper()):
    #     agingatlas_genes_symbol_transfer[RNAdict_upper.get(i.upper())[0]]='https://ngdc.cncb.ac.cn/aging/age_related_genes'
    #     print('RNA')

In [659]:
# already check that one to one
# already check that RNA only TERC
agingatlas_genes_symbol=set()
agingatlas_genes_symbol_transfer={}
for i in agingatlas_genes_list:
    if Genedict_upper.get(i.upper()):
        agingatlas_genes_symbol_transfer[Genedict_upper.get(i.upper())[0]]=i
    elif RNAdict_upper.get(i.upper()):
        agingatlas_genes_symbol_transfer[RNAdict_upper.get(i.upper())[0]]=i

In [754]:
for i in Gene_all:
    if agingatlas_genes_symbol_transfer.get(i.get('entity')):
        # i['AgingAtlas_url']='https://ngdc.cncb.ac.cn/aging/age_related_genes'
        i['AgingAtlas_url']='https://ngdc.cncb.ac.cn/aging/age_related_genes?species=Homo%20sapiens&data='+agingatlas_genes_symbol_transfer.get(i.get('entity'))
        i['AgingAtlas_symbol']=agingatlas_genes_symbol_transfer.get(i.get('entity'))
    else:
        i['AgingAtlas_url']=''
        i['AgingAtlas_symbol']=''

In [780]:
all_es=[]
mu=0
di=0
noturl=[]
for i in Gene_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'Also_known_as':i.get('Also_known_as'),'Official_Full_Name':i.get('Official_Full_Name'),'Summary':sent_tokenize(i.get('Summary'))[0],'NCBI_URL':i.get('NCBI_URL'),'NCBI_ID':i.get('NCBI_ID'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-','')),'NCBI_URL':i.get('NCBI_URL'),'GenAge_URL':i.get('GenAge_url'),'GenAge_symbol':i.get('GenAge_symbol'),'LongevityMap_URL':i.get('LongevityMap_url'),'LongevityMap_symbol':i.get('LongevityMap_symbol'),'AgingBank_URL':i.get('AgingBank_url'),'AgingBank_symbol':i.get('AgingBank_symbol'),'AgingAtlas_URL':i.get('AgingAtlas_url'),'AgingAtlas_symbol':i.get('AgingAtlas_symbol')})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'Also_known_as':i.get('Also_known_as'),'Official_Full_Name':i.get('Official_Full_Name'),'Summary':'','NCBI_URL':i.get('NCBI_URL'),'NCBI_ID':i.get('NCBI_ID'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-','')),'GenAge_URL':i.get('GenAge_url'),'GenAge_symbol':i.get('GenAge_symbol'),'LongevityMap_URL':i.get('LongevityMap_url'),'LongevityMap_symbol':i.get('LongevityMap_symbol'),'AgingBank_URL':i.get('AgingBank_url'),'AgingBank_symbol':i.get('AgingBank_symbol'),'AgingAtlas_URL':i.get('AgingAtlas_url'),'AgingAtlas_symbol':i.get('AgingAtlas_symbol')})
for i in RNA_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'specific_type':i.get('specific_type'),'Also_known_as':i.get('Also_known_as'),'Official_Full_Name':i.get('Official_Full_Name'),'Summary':sent_tokenize(i.get('Summary'))[0],'NCBI_URL':i.get('NCBI_URL'),'NCBI_ID':i.get('NCBI_ID'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-','')),'GenAge_URL':i.get('GenAge_url'),'GenAge_symbol':i.get('GenAge_symbol'),'LongevityMap_URL':i.get('LongevityMap_url'),'LongevityMap_symbol':i.get('LongevityMap_symbol'),'AgingBank_URL':i.get('AgingBank_url'),'AgingBank_symbol':i.get('AgingBank_symbol'),'AgingAtlas_URL':i.get('AgingAtlas_url'),'AgingAtlas_symbol':i.get('AgingAtlas_symbol')})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'specific_type':i.get('specific_type'),'Also_known_as':i.get('Also_known_as'),'Official_Full_Name':i.get('Official_Full_Name'),'Summary':'','NCBI_URL':i.get('NCBI_URL'),'NCBI_ID':i.get('NCBI_ID'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-','')),'GenAge_URL':i.get('GenAge_url'),'GenAge_symbol':i.get('GenAge_symbol'),'LongevityMap_URL':i.get('LongevityMap_url'),'LongevityMap_symbol':i.get('LongevityMap_symbol'),'AgingBank_URL':i.get('AgingBank_url'),'AgingBank_symbol':i.get('AgingBank_symbol'),'AgingAtlas_URL':i.get('AgingAtlas_url'),'AgingAtlas_symbol':i.get('AgingAtlas_symbol')})
for i in Carbohydrate_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':sent_tokenize(i.get('Summary'))[0],'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':'','url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in Lipid_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':sent_tokenize(i.get('Summary'))[0],'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':'','url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in Peptide_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':sent_tokenize(i.get('Summary'))[0],'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':'','url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in Pharmaceutical_Preparations_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':sent_tokenize(i.get('Summary'))[0],'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':'PharmaceuticalPreparations'+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':'','url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':'PharmaceuticalPreparations'+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in Protein_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':sent_tokenize(i.get('Summary'))[0],'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':'','url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in Toxin_all:
    if i.get('Summary'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':sent_tokenize(i.get('Summary'))[0],'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'Summary':'','url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
for i in Mutation_all:
    if i.get('Genomic View'):
        all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'Genomic_View':i.get('Genomic View'),'Alleles':i.get('Alleles'),'Position':i.get('Position'),'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        mu+=1
for i in Disease_all:
    if i.get('url'):
        if i.get('Summary'):
            all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'Summary':sent_tokenize(i.get('Summary'))[0],'_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
        else:
            all_es.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'literature':len(i.get('PMID')),'targetshapes':i.get('targetshapes'),'allyears':i.get('allyears'),'type':i.get('type'),'MeshID':i.get('MESH'),'url':'https://www.ncbi.nlm.nih.gov'+i.get('url'),'Summary':'','_id':i.get('type')+'_'+i.get('entity'),'datest':int(i.get('datestring')[0].replace('-',''))})
    else:
        di+=1
        noturl.append(i)

In [781]:
all_es_copy=copy.deepcopy(all_es)

In [782]:
len(all_es_copy)

12283

In [783]:
for i in all_es:
    i['allwordsdict']={}
    if i.get('Also_known_as'):
        i['allwords']=i.get('allwords',[])+i.get('Also_known_as').split('; ')
        for l in i.get('Also_known_as').split('; '):
            i['allwordsdict'][l.upper()]=l
    for j in i.get('targetshapes'):
        if j.upper() != i.get('entity').upper() and not i.get('allwordsdict').get(j.upper()):
            i['allwords']=i.get('allwords',[])+[j.upper()]
            i['allwordsdict'][j.upper()]=j
    if i.get('targetshapes'):
        del i['targetshapes']
    del i['allwordsdict']

In [784]:
all_es[-1000]

{'entity': 'Microphthalmos',
 'PMID': ['29100093', '11585313', '17603803', '33110058', '3523475'],
 'literature': 5,
 'allyears': [2020, 2017, 2007, 2001, 1986],
 'type': 'Disease',
 'MeshID': 'D008850',
 'url': 'https://www.ncbi.nlm.nih.gov/mesh/68008850',
 'Summary': 'Congenital or developmental anomaly in which the eyeballs are abnormally small.',
 '_id': 'Disease_Microphthalmos',
 'datest': 20201028,
 'allwords': ['MICROPHTHALMIA']}

In [785]:
len(all_es)

12283

In [786]:
noturls=[]
for i in Disease_all:
    if not i.get('url'):
        noturls.append(i)

In [787]:
len(noturls)

16404

In [788]:
len(Disease_all)

19902

In [789]:
json.dump(all_es,open('results/step2/all_es_lack.json','w'))

In [790]:
entity_pairs=['Gene','RNA','Carbohydrate','Lipid','Peptide', 'Pharmaceutical_Preparations', 'Protein', 'Toxin','Mutation', 'Disease']

In [791]:
all_es_result=[]
for j in entity_pairs:
    for i in names[j+'_all']:
        if i.get('specific_type'):
            all_es_result.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'target':i.get('target'),'date':i.get('date'),'type':i.get('specific_type'),'sentence':i.get('sentence'),'_id':i.get('type')+'_'+i.get('entity')})
        else:
            if i.get('type')=='Disease':
                if i.get('url'):
                    all_es_result.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'target':i.get('target'),'date':i.get('date'),'type':i.get('type'),'sentence':i.get('sentence'),'_id':i.get('type')+'_'+i.get('entity')})
            elif i.get('type')=='Mutation':
                if i.get('Genomic View'):
                    all_es_result.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'target':i.get('target'),'date':i.get('date'),'type':i.get('type'),'sentence':i.get('sentence'),'_id':i.get('type')+'_'+i.get('entity')})
            else:
                all_es_result.append({'entity':i.get('entity'),'PMID':i.get('PMID'),'target':i.get('target'),'date':i.get('date'),'type':i.get('type'),'sentence':i.get('sentence'),'_id':i.get('type')+'_'+i.get('entity')})

In [792]:
jcr=pd.read_csv("../JCR/jcr2022.tsv",sep="\t")

In [793]:
collections_pubtator_info = db.pubtator_info

In [794]:
pubtator_info=list(collections_pubtator_info.find())

In [795]:
pubtator_info_dict={}
for i in pubtator_info:
    pubtator_info_dict[i.get('PMID')]=i

In [796]:
jt2if_all={}
for i in range(len(jcr)):
    if jt2if_all.get(jcr.iloc[i,1].lower()):
        if jt2if_all.get(jcr.iloc[i,1].lower())==0.0:
            jt2if_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
    else:
        jt2if_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
jt2if_all['aging']=jt2if_all.get('aging-us')

In [797]:
jt2if_brief={}
for i in range(len(jcr)):
    if jt2if_brief.get(jcr.iloc[i,2].lower()):
        if jt2if_brief.get(jcr.iloc[i,2].lower())==0.0:
            jt2if_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
    else:
        jt2if_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
jt2if_brief['aging']=jt2if_brief.get('aging-us')

In [798]:
jt2if5_all={}
for i in range(len(jcr)):
    if jt2if5_all.get(jcr.iloc[i,1].lower()):
        if jt2if5_all.get(jcr.iloc[i,1].lower())==0.0:
            jt2if5_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
    else:
        jt2if5_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,9]
jt2if5_all['aging']=jt2if5_all.get('aging-us')

In [799]:
jt2if5_brief={}
for i in range(len(jcr)):
    
    if jt2if5_brief.get(jcr.iloc[i,2].lower()):
        if jt2if5_brief.get(jcr.iloc[i,2].lower())==0.0:
            jt2if5_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
    else:
        jt2if5_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,9]
jt2if5_brief['aging']=jt2if5_brief.get('aging-us')

In [800]:
for i in all_es_result:
    iflist=[]
    if5list=[]
    TAlist=[]
    JTlist=[]
    yearlist=[]
    for j in i.get('PMID'):
        if jt2if_brief.get(pubtator_info_dict.get(j).get('TA').lower()):
            iflist.append(jt2if_brief.get(pubtator_info_dict.get(j).get('TA').lower()))
        elif jt2if_all.get(pubtator_info_dict.get(j).get('JT').lower()):
            iflist.append(jt2if_all.get(pubtator_info_dict.get(j).get('JT').lower()))
        else:
            iflist.append(0.0)
        if jt2if5_brief.get(pubtator_info_dict.get(j).get('TA').lower()):
            if5list.append(jt2if5_brief.get(pubtator_info_dict.get(j).get('TA').lower()))
        elif jt2if5_all.get(pubtator_info_dict.get(j).get('JT').lower()):
            if5list.append(jt2if5_all.get(pubtator_info_dict.get(j).get('JT').lower()))
        else:
            if5list.append(0.0)
        TAlist.append(pubtator_info_dict.get(j).get('TA'))
        JTlist.append(pubtator_info_dict.get(j).get('JT'))
    for l in i.get('date'):
        yearlist.append(int(str(l)[:4]))
    i['IF']=iflist
    i['IF5']=if5list
    i['TA']=TAlist
    i['JT']=JTlist
    i['year']=yearlist

In [801]:
json.dump(all_es_result,open('results/step2/all_es_result.json','w'))

In [802]:
len(all_es_result)

12283

In [803]:
blank=[{'Module':'','Description':'','Entity_Relation':'','Username':'','Email':''}]

In [685]:
json.dump(blank,open('results/step2/feedback.json','w'))